In [ ]:
!pip install xgboost --quiet

import pandas as pd
import numpy as np
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
from datetime import timedelta


train = pd.read_csv('train.csv')

if 'Unnamed: 0' in train.columns:
    train = train.drop(columns=['Unnamed: 0'])

def build_datetime(df):
    df = df.rename(columns={'min': 'minute'})
    dt = pd.to_datetime(df[['year', 'month', 'day', 'hour', 'minute']], errors='coerce') + pd.to_timedelta(df['sec'], unit='s')
    df = df.rename(columns={'minute': 'min'})

train['main_dt'] = build_datetime(train)
after_cols = {
    'year_as': 'year',
    'month_as': 'month',
    'day_as': 'day',
    'hour_as': 'hour',
    'min_as': 'minute'
}
after_cols = ['year_as', 'month_as', 'day_as', 'hour_as', 'min_as']
after_df = train[after_cols].copy()
after_df.columns = ['year', 'month', 'day', 'hour', 'minute']

train['after_dt'] = pd.to_datetime(after_df, errors='coerce') + pd.to_timedelta(train['sec_as'], unit='s')

train['delta_seconds'] = (train['after_dt'] - train['main_dt']).dt.total_seconds()

features = ['year', 'month', 'day', 'hour', 'min', 'sec', 'lat', 'lon', 'depth', 'class']
target = ['delta_seconds', 'lat_as', 'lon_as', 'depth_as', 'class_as']

X = train[features]
y = train[target]

model = MultiOutputRegressor(XGBRegressor(n_estimators=100, random_state=42))
model.fit(X, y)
y_pred = model.predict(X)

mae_list = []
for i, col in enumerate(target):
    mae = mean_absolute_error(y[col], y_pred[:, i])
    mae_list.append((col, mae))

for name, value in mae_list:
    print(f"{name} MAE: {value:.4f}")

mean_mae = np.mean([mae for _, mae in mae_list])
print(f"\n Средний MAE: {mean_mae:.4f}")


delta_seconds MAE: 12498.7239
lat_as MAE: 0.0051
lon_as MAE: 0.0075
depth_as MAE: 0.2347
class_as MAE: 0.0522

🔎 Средний MAE: 2499.8047


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
import lightgbm as lgb

train = pd.read_csv("train.csv")
if 'Unnamed: 0' in train.columns:
    train.drop(columns=['Unnamed: 0'], inplace=True)

def build_datetime(df):
    df = df.rename(columns={'min': 'minute'})
    dt = pd.to_datetime(df[['year', 'month', 'day', 'hour', 'minute']], errors='coerce') + pd.to_timedelta(df['sec'], unit='s')
    df = df.rename(columns={'minute': 'min'})
    return dt

train['main_dt'] = build_datetime(train)

after_time = train[['year_as', 'month_as', 'day_as', 'hour_as', 'min_as', 'sec_as']].copy()
after_time = after_time.rename(columns={
    'year_as': 'year',
    'month_as': 'month',
    'day_as': 'day',
    'hour_as': 'hour',
    'min_as': 'minute'
})
train['after_dt'] = pd.to_datetime(after_time[['year', 'month', 'day', 'hour', 'minute']], errors='coerce') + pd.to_timedelta(after_time['sec_as'], unit='s')

train['delta_seconds'] = (train['after_dt'] - train['main_dt']).dt.total_seconds()

train = train[train['delta_seconds'] > 0].copy()
train['log_delta'] = np.log1p(train['delta_seconds'])

features = ['year', 'month', 'day', 'hour', 'min', 'sec', 'lat', 'lon', 'depth', 'class']
X = train[features]
y_log = train['log_delta']

kf = KFold(n_splits=5, shuffle=True, random_state=42)
mae_scores = []

for fold, (train_idx, valid_idx) in enumerate(kf.split(X)):
    X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
    y_train, y_valid = y_log.iloc[train_idx], y_log.iloc[valid_idx]

    model = lgb.LGBMRegressor(
        n_estimators=1000,
        learning_rate=0.03,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1
    )

    model.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        callbacks=[
            lgb.early_stopping(stopping_rounds=50),
            lgb.log_evaluation(period=100)
        ]
    )

    y_pred = model.predict(X_valid)
    y_valid_exp = np.expm1(y_valid)
    y_pred_exp = np.expm1(y_pred)

    score = mean_absolute_error(y_valid_exp, y_pred_exp)
    mae_scores.append(score)

    print(f" Fold {fold + 1} MAE: {score:.4f}")

print(f"\n Средний MAE по 5 фолдам: {np.mean(mae_scores):.4f}")


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000734 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 984
[LightGBM] [Info] Number of data points in the train set: 989, number of used features: 10
[LightGBM] [Info] Start training from score 10.212627
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 50 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM]

In [ ]:
!pip install lightgbm --quiet

In [ ]:
# ✅ PyTorch Model for Aftershock Prediction (MultiOutput Regression)
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
import pandas as pd
import numpy as np

# === 1. Подготовка данных ===
features = [
    "year", "month", "day", "hour", "min", "sec",
    "lat", "lon", "depth", "class",
    "weekday", "is_weekend", "hour_fraction", "is_night", "season",
    "delta_lat", "delta_lon", "distance_km",
    "log_class", "log_depth", "lat_bin", "lon_bin",
    "lat_times_lon", "depth_div_class"
]
targets = [
    "year_as", "month_as", "day_as", "hour_as", "min_as", "sec_as",
    "lat_as", "lon_as", "depth_as", "class_as"
]

X = train[features].copy()
y = train[targets].copy()

mask = X.notna().all(axis=1) & y.notna().all(axis=1)
X = X[mask]
y = y[mask]

scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)

X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y_scaled, dtype=torch.float32)

dataset = TensorDataset(X_tensor, y_tensor)
train_ds, val_ds = random_split(dataset, [int(0.8 * len(dataset)), len(dataset) - int(0.8 * len(dataset))])
train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=64, shuffle=False)

# === 2. Модель ===
class AftershockModel(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, output_size)
        )

    def forward(self, x):
        return self.net(x)

model = AftershockModel(input_size=X_tensor.shape[1], output_size=y_tensor.shape[1])

# === 3. Обучение ===
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()

for epoch in range(50):
    model.train()
    total_loss = 0
    for xb, yb in train_loader:
        pred = model(xb)
        loss = loss_fn(pred, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * xb.size(0)

    val_loss = 0
    model.eval()
    with torch.no_grad():
        for xb, yb in val_loader:
            pred = model(xb)
            loss = loss_fn(pred, yb)
            val_loss += loss.item() * xb.size(0)

    print(f"Epoch {epoch+1:2d} | Train Loss: {total_loss / len(train_loader.dataset):.4f} | Val Loss: {val_loss / len(val_loader.dataset):.4f}")

# === 4. Предсказание ===
X_test = test[features].copy().fillna(0)
X_test_scaled = scaler_X.transform(X_test)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)

model.eval()
with torch.no_grad():
    test_preds_scaled = model(X_test_tensor).numpy()
    test_preds = scaler_y.inverse_transform(test_preds_scaled)

for i, col in enumerate(targets):
    test[col] = test_preds[:, i]

# === 5. Сборка submission ===
test["year_as"] = test["year_as"].round().astype(int)
test["month_as"] = test["month_as"].round().astype(int)
test["day_as"] = test["day_as"].round().astype(int)
test["hour_as"] = test["hour_as"].round().astype(int)
test["min_as"] = test["min_as"].round().astype(int)
test["sec_as"] = test["sec_as"].round(3)
test["lat_as"] = test["lat_as"].round(5)
test["lon_as"] = test["lon_as"].round(5)
test["depth_as"] = test["depth_as"].round(2)
test["class_as"] = test["class_as"].round(2)

submission = test[[
    "id_eq", "year_as", "month_as", "day_as", "hour_as", "min_as", "sec_as",
    "lat_as", "lon_as", "depth_as", "class_as"
]]
submission.to_csv("submission_pytorch.csv", index=False)
print("\n✅ submission_pytorch.csv сохранён!")

NameError: name 'train' is not defined

In [ ]:
!pip install pytorch-tabnet

In [ ]:
from pytorch_tabnet.regression import TabNetRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
import pandas as pd
import numpy as np
import torch

np.random.seed(42)
torch.manual_seed(42)


features = [
    "year", "month", "day", "hour", "min", "sec",
    "lat", "lon", "depth", "class",
    "weekday", "is_weekend", "hour_fraction", "is_night", "season",
    "delta_lat", "delta_lon", "distance_km",
    "log_class", "log_depth", "lat_bin", "lon_bin",
    "lat_times_lon", "depth_div_class"
]
targets = [
    "year_as", "month_as", "day_as", "hour_as", "min_as", "sec_as",
    "lat_as", "lon_as", "depth_as", "class_as"
]

X = train[features].replace([np.inf, -np.inf], np.nan).fillna(0).copy()
y = train[targets].replace([np.inf, -np.inf], np.nan).fillna(0).copy()

scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)

kf = KFold(n_splits=5, shuffle=True, random_state=42)
mae_list = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X_scaled)):
    print(f"\n Fold {fold+1}")

    X_train, X_val = X_scaled[train_idx], X_scaled[val_idx]
    y_train, y_val = y_scaled[train_idx], y_scaled[val_idx]

    model = TabNetRegressor(
        n_d=64, n_a=64, n_steps=5,
        optimizer_fn=torch.optim.Adam,
        optimizer_params=dict(lr=2e-3),
        mask_type="entmax",
        seed=42, verbose=0
    )

    model.fit(
        X_train=X_train, y_train=y_train,
        eval_set=[(X_val, y_val)],
        eval_metric=["mae"],
        max_epochs=100,
        patience=10,
        batch_size=256,
        virtual_batch_size=128
    )

    preds_val = model.predict(X_val)
    preds_val_inv = scaler_y.inverse_transform(preds_val)
    y_val_inv = scaler_y.inverse_transform(y_val)

    fold_maes = [mean_absolute_error(y_val_inv[:, i], preds_val_inv[:, i]) for i in range(len(targets))]
    for t, m in zip(targets, fold_maes):
        print(f"{t}: {m:.4f}")
    print(f"🔁 Fold MAE: {np.mean(fold_maes):.4f}")
    mae_list.append(np.mean(fold_maes))

print(f"\n Mean MAE across folds: {np.mean(mae_list):.4f}")


ModuleNotFoundError: No module named 'pytorch_tabnet.regression'

In [ ]:
!pip install pytorch-tabnet==3.1.1
!pip install category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.4/887.4 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: torch
    Found existing installation: torch 2.6.0+cu124
    Uninstalling torch-2.6.0+cu124:
      Successfully uninstalled torch-2.6.0+cu124
  Attempting uninstall: pytorch-tabnet
    Found existing installation: pytorch-tabnet 4.1.0
    Uninstalling pytorch-tabnet-4.

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 2.4 MB/s eta 0:00:00
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 447, in run
^C


In [ ]:
from google.colab import files
import numpy as np
import pandas as pd
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")


In [ ]:
def add_features(df):
    df['datetime'] = pd.to_datetime(dict(
        year=df['year'], month=df['month'], day=df['day'],
        hour=df['hour'], minute=df['min'], second=df['sec']
    ), errors='coerce')

    df['weekday'] = df['datetime'].dt.weekday.fillna(0)
    df['is_weekend'] = df['weekday'].isin([5, 6]).astype(int)
    df['hour_fraction'] = df['hour'] + df['min'] / 60 + df['sec'] / 3600
    df['is_night'] = df['hour'].between(0, 6).astype(int)
    df['season'] = ((df['month'] % 12 + 3) // 3).astype(int)

    df['delta_lat'] = df['lat'] - df['lat'].mean()
    df['delta_lon'] = df['lon'] - df['lon'].mean()
    df['distance_km'] = np.sqrt(df['delta_lat']**2 + df['delta_lon']**2) * 111

    df['log_class'] = np.log1p(df['class'])
    df['log_depth'] = np.log1p(df['depth'])

    df['lat_bin'] = pd.cut(df['lat'], bins=10, labels=False)
    df['lon_bin'] = pd.cut(df['lon'], bins=10, labels=False)

    df['lat_times_lon'] = df['lat'] * df['lon']
    df['depth_div_class'] = df['depth'] / (df['class'] + 1)

    return df.fillna(0)

train = add_features(train)
test = add_features(test)

/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [ ]:
features = [
    "year", "month", "day", "hour", "min", "sec",
    "lat", "lon", "depth", "class",
    "weekday", "is_weekend", "hour_fraction", "is_night", "season",
    "delta_lat", "delta_lon", "distance_km",
    "log_class", "log_depth", "lat_bin", "lon_bin",
    "lat_times_lon", "depth_div_class"
]

targets = [
    "year_as", "month_as", "day_as", "hour_as", "min_as", "sec_as",
    "lat_as", "lon_as", "depth_as", "class_as"
]


In [ ]:
from sklearn.preprocessing import StandardScaler
from pytorch_tabnet.regression import TabNetRegressor
import torch

X = train[features].replace([np.inf, -np.inf], 0).fillna(0)
y = train[targets].replace([np.inf, -np.inf], 0).fillna(0)

scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)

tabnet = TabNetRegressor(
    n_d=64, n_a=64, n_steps=5,
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=2e-3),
    mask_type="entmax",
    seed=42, verbose=1
)

tabnet.fit(
    X_train=X_scaled, y_train=y_scaled,
    max_epochs=100,
    patience=15,
    batch_size=256,
    virtual_batch_size=128
)


ModuleNotFoundError: No module named 'pytorch_tabnet.regression'

In [ ]:
!pip uninstall -y pytorch-tabnet
!pip install pytorch-tabnet==3.1.1


Found existing installation: pytorch-tabnet 3.1.1
Uninstalling pytorch-tabnet-3.1.1:
  Successfully uninstalled pytorch-tabnet-3.1.1
  Using cached pytorch_tabnet-3.1.1-py3-none-any.whl.metadata (12 kB)
Using cached pytorch_tabnet-3.1.1-py3-none-any.whl (39 kB)


In [ ]:
from pytorch_tabnet.regression import TabNetRegressor

ModuleNotFoundError: No module named 'pytorch_tabnet.regression'

In [ ]:
!pip uninstall -y pytorch-tabnet tabnet

Found existing installation: pytorch-tabnet 3.1.1
Uninstalling pytorch-tabnet-3.1.1:
  Successfully uninstalled pytorch-tabnet-3.1.1


In [ ]:
!pip install torch==1.9.0
!pip install pytorch-tabnet==3.1.1


ERROR: Could not find a version that satisfies the requirement torch==1.9.0 (from versions: 1.13.0, 1.13.1, 2.0.0, 2.0.1, 2.1.0, 2.1.1, 2.1.2, 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1, 2.5.0, 2.5.1, 2.6.0)
ERROR: No matching distribution found for torch==1.9.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.4/887.4 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: t

In [ ]:
from pytorch_tabnet.regression import TabNetRegressor

ModuleNotFoundError: No module named 'pytorch_tabnet.regression'

In [ ]:
from pytorch_tabnet.tab_model import TabNetRegressor

In [ ]:
from google.colab import files
import pandas as pd
import numpy as np
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")


In [ ]:
def add_features(df):
    df['datetime'] = pd.to_datetime(dict(
        year=df['year'], month=df['month'], day=df['day'],
        hour=df['hour'], minute=df['min'], second=df['sec']
    ), errors='coerce')

    df['weekday'] = df['datetime'].dt.weekday.fillna(0)
    df['is_weekend'] = df['weekday'].isin([5, 6]).astype(int)
    df['hour_fraction'] = df['hour'] + df['min'] / 60 + df['sec'] / 3600
    df['is_night'] = df['hour'].between(0, 6).astype(int)
    df['season'] = ((df['month'] % 12 + 3) // 3).astype(int)

    df['delta_lat'] = df['lat'] - df['lat'].mean()
    df['delta_lon'] = df['lon'] - df['lon'].mean()
    df['distance_km'] = np.sqrt(df['delta_lat']**2 + df['delta_lon']**2) * 111

    df['log_class'] = np.log1p(df['class'])
    df['log_depth'] = np.log1p(df['depth'])

    df['lat_bin'] = pd.cut(df['lat'], bins=10, labels=False)
    df['lon_bin'] = pd.cut(df['lon'], bins=10, labels=False)

    df['lat_times_lon'] = df['lat'] * df['lon']
    df['depth_div_class'] = df['depth'] / (df['class'] + 1)

    return df.fillna(0)

train = add_features(train)
test = add_features(test)


/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [ ]:
features = [
    "year", "month", "day", "hour", "min", "sec",
    "lat", "lon", "depth", "class",
    "weekday", "is_weekend", "hour_fraction", "is_night", "season",
    "delta_lat", "delta_lon", "distance_km",
    "log_class", "log_depth", "lat_bin", "lon_bin",
    "lat_times_lon", "depth_div_class"
]

targets = [
    "year_as", "month_as", "day_as", "hour_as", "min_as", "sec_as",
    "lat_as", "lon_as", "depth_as", "class_as"
]


In [ ]:
from sklearn.preprocessing import StandardScaler
from pytorch_tabnet.tab_model import TabNetRegressor
import torch

X = train[features].replace([np.inf, -np.inf], 0).fillna(0)
y = train[targets].replace([np.inf, -np.inf], 0).fillna(0)

scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)

tabnet = TabNetRegressor(
    n_d=64, n_a=64, n_steps=5,
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=2e-3),
    mask_type="entmax",
    seed=42, verbose=1
)

tabnet.fit(
    X_train=X_scaled, y_train=y_scaled,
    max_epochs=100,
    patience=15,
    batch_size=256,
    virtual_batch_size=128
)


Device used : cpu
No early stopping will be performed, last training weights will be used.
epoch 0  | loss: 14.68044|  0:00:00s
epoch 1  | loss: 8.41692 |  0:00:01s
epoch 2  | loss: 5.38555 |  0:00:02s
epoch 3  | loss: 3.70667 |  0:00:02s
epoch 4  | loss: 2.74298 |  0:00:03s
epoch 5  | loss: 2.19216 |  0:00:03s
epoch 6  | loss: 1.90135 |  0:00:04s
epoch 7  | loss: 1.6307  |  0:00:04s
epoch 8  | loss: 1.47917 |  0:00:05s
epoch 9  | loss: 1.36511 |  0:00:06s
epoch 10 | loss: 1.19603 |  0:00:06s
epoch 11 | loss: 1.1415  |  0:00:07s
epoch 12 | loss: 1.06312 |  0:00:07s
epoch 13 | loss: 1.00338 |  0:00:08s
epoch 14 | loss: 0.96518 |  0:00:09s
epoch 15 | loss: 0.91795 |  0:00:10s
epoch 16 | loss: 0.90576 |  0:00:11s
epoch 17 | loss: 0.87111 |  0:00:11s
epoch 18 | loss: 0.84798 |  0:00:12s
epoch 19 | loss: 0.85314 |  0:00:12s
epoch 20 | loss: 0.82235 |  0:00:13s
epoch 21 | loss: 0.82169 |  0:00:14s
epoch 22 | loss: 0.77426 |  0:00:14s
epoch 23 | loss: 0.7372  |  0:00:15s
epoch 24 | loss: 0.72

In [ ]:
X_test = test[features].replace([np.inf, -np.inf], 0).fillna(0)
X_test_scaled = scaler_X.transform(X_test)

preds_scaled = tabnet.predict(X_test_scaled)
preds = scaler_y.inverse_transform(preds_scaled)

for i, col in enumerate(targets):
    test[col] = preds[:, i]

test["year_as"] = test["year_as"].round().astype(int)
test["month_as"] = test["month_as"].round().astype(int)
test["day_as"] = test["day_as"].round().astype(int)
test["hour_as"] = test["hour_as"].round().astype(int)
test["min_as"] = test["min_as"].round().astype(int)
test["sec_as"] = test["sec_as"].round(3)
test["lat_as"] = test["lat_as"].round(5)
test["lon_as"] = test["lon_as"].round(5)
test["depth_as"] = test["depth_as"].round(2)
test["class_as"] = test["class_as"].round(2)

submission = test[[
    "id_eq", "year_as", "month_as", "day_as", "hour_as", "min_as", "sec_as",
    "lat_as", "lon_as", "depth_as", "class_as"
]]

submission.to_csv("submission_tabnet.csv", index=False)

from google.colab import files
files.download("submission_tabnet.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
import optuna
import torch

X = train[features].replace([np.inf, -np.inf], 0).fillna(0)
y = train[targets].replace([np.inf, -np.inf], 0).fillna(0)

scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)


ModuleNotFoundError: No module named 'optuna'

In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 16.0 MB/s eta 0:00:00


In [ ]:
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
import optuna
import torch

X = train[features].replace([np.inf, -np.inf], 0).fillna(0)
y = train[targets].replace([np.inf, -np.inf], 0).fillna(0)

scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)

In [ ]:
def objective(trial):
    n_d = trial.suggest_int("n_d", 16, 128)
    n_a = trial.suggest_int("n_a", 16, 128)
    n_steps = trial.suggest_int("n_steps", 3, 10)
    gamma = trial.suggest_float("gamma", 1.0, 2.0)
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    dropout = trial.suggest_float("dropout", 0.0, 0.4)

    kf = KFold(n_splits=3, shuffle=True, random_state=42)
    maes = []

    for train_idx, val_idx in kf.split(X_scaled):
        model = TabNetRegressor(
            n_d=n_d, n_a=n_a, n_steps=n_steps, gamma=gamma,
            optimizer_fn=torch.optim.Adam,
            optimizer_params=dict(lr=lr),
            mask_type='entmax',
            n_shared=1, n_independent=1,
            seed=42,
            verbose=0
        )

        model.fit(
            X_scaled[train_idx], y_scaled[train_idx],
            eval_set=[(X_scaled[val_idx], y_scaled[val_idx])],
            patience=15, max_epochs=200,
            batch_size=512,
            virtual_batch_size=128,
            drop_last=False,
        )

        preds = model.predict(X_scaled[val_idx])
        preds = scaler_y.inverse_transform(preds)
        true = scaler_y.inverse_transform(y_scaled[val_idx])

        mae = mean_absolute_error(true, preds)
        maes.append(mae)

    return sum(maes) / len(maes)

In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)

print(" Best params:", study.best_params)

[I 2025-04-13 00:25:42,555] A new study created in memory with name: no-name-6ad781f8-328b-4736-a433-4873b5f761ed



Early stopping occurred at epoch 177 with best_epoch = 162 and best_val_0_mse = 1.00954
Best weights from best epoch are automatically used!
Stop training because you reached max_epochs = 200 with best_epoch = 195 and best_val_0_mse = 0.83594
Best weights from best epoch are automatically used!


[I 2025-04-13 00:28:59,872] Trial 0 finished with value: 21.65082466334975 and parameters: {'n_d': 80, 'n_a': 81, 'n_steps': 8, 'gamma': 1.6817180905654234, 'lr': 0.0007458434215523452, 'dropout': 0.07820590240573787}. Best is trial 0 with value: 21.65082466334975.


Stop training because you reached max_epochs = 200 with best_epoch = 199 and best_val_0_mse = 0.8973
Best weights from best epoch are automatically used!

Early stopping occurred at epoch 140 with best_epoch = 125 and best_val_0_mse = 1.02427
Best weights from best epoch are automatically used!
Stop training because you reached max_epochs = 200 with best_epoch = 187 and best_val_0_mse = 0.81618
Best weights from best epoch are automatically used!


[I 2025-04-13 00:31:26,518] Trial 1 finished with value: 21.47994881075314 and parameters: {'n_d': 76, 'n_a': 40, 'n_steps': 9, 'gamma': 1.6874261129914747, 'lr': 0.0012835035143016515, 'dropout': 0.12383097146646582}. Best is trial 1 with value: 21.47994881075314.


...


Early stopping occurred at epoch 119 with best_epoch = 104 and best_val_0_mse = 0.69479
Best weights from best epoch are automatically used!

Early stopping occurred at epoch 70 with best_epoch = 55 and best_val_0_mse = 0.72579
Best weights from best epoch are automatically used!

Early stopping occurred at epoch 131 with best_epoch = 116 and best_val_0_mse = 0.69106
Best weights from best epoch are automatically used!


[I 2025-04-13 00:50:02,216] Trial 18 finished with value: 20.169613866340498 and parameters: {'n_d': 60, 'n_a': 93, 'n_steps': 5, 'gamma': 1.5869485705661934, 'lr': 0.006100876875808974, 'dropout': 0.3886300425191891}. Best is trial 10 with value: 19.10319782116517.



Early stopping occurred at epoch 78 with best_epoch = 63 and best_val_0_mse = 0.66885
Best weights from best epoch are automatically used!

Early stopping occurred at epoch 99 with best_epoch = 84 and best_val_0_mse = 0.68785
Best weights from best epoch are automatically used!

Early stopping occurred at epoch 95 with best_epoch = 80 and best_val_0_mse = 0.65742
Best weights from best epoch are automatically used!


[I 2025-04-13 00:51:25,207] Trial 19 finished with value: 19.740055994413638 and parameters: {'n_d': 25, 'n_a': 128, 'n_steps': 6, 'gamma': 1.2518993151161117, 'lr': 0.002946485837287534, 'dropout': 0.19018841483279517}. Best is trial 10 with value: 19.10319782116517.



Early stopping occurred at epoch 111 with best_epoch = 96 and best_val_0_mse = 0.68127
Best weights from best epoch are automatically used!
✅ Best params: {'n_d': 16, 'n_a': 123, 'n_steps': 3, 'gamma': 1.5150385312401526, 'lr': 0.009840625231932706, 'dropout': 0.0033641581317567226}


In [ ]:
import pandas as pd
import numpy as np

# Загрузка
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

def add_features(df):
    df["datetime"] = pd.to_datetime(dict(
        year=df["year"], month=df["month"], day=df["day"],
        hour=df["hour"], minute=df["min"], second=df["sec"]
    ), errors='coerce')
    df["weekday"] = df["datetime"].dt.weekday.fillna(0).astype(int)
    df["is_weekend"] = df["weekday"].isin([5, 6]).astype(int)
    df["hour_fraction"] = df["hour"] + df["min"] / 60 + df["sec"] / 3600
    df["is_night"] = ((df["hour"] < 6) | (df["hour"] > 22)).astype(int)
    df["season"] = df["month"] % 12 // 3  # 0=Winter, 1=Spring, etc.
    df["log_class"] = np.log1p(df["class"])
    df["log_depth"] = np.log1p(df["depth"])
    df["lat_bin"] = pd.cut(df["lat"], bins=10, labels=False)
    df["lon_bin"] = pd.cut(df["lon"], bins=10, labels=False)
    df["lat_times_lon"] = df["lat"] * df["lon"]
    df["depth_div_class"] = df["depth"] / (df["class"] + 1)
    df["delta_lat"] = df["lat"] - df.get("lat_as", df["lat"])
    df["delta_lon"] = df["lon"] - df.get("lon_as", df["lon"])
    df["distance_km"] = np.sqrt(df["delta_lat"]**2 + df["delta_lon"]**2) * 111
    df = df.replace([np.inf, -np.inf], 0).fillna(0)
    return df

train = add_features(train)
test = add_features(test)

features = [
    "year", "month", "day", "hour", "min", "sec",
    "lat", "lon", "depth", "class",
    "weekday", "is_weekend", "hour_fraction", "is_night", "season",
    "delta_lat", "delta_lon", "distance_km",
    "log_class", "log_depth", "lat_bin", "lon_bin",
    "lat_times_lon", "depth_div_class"
]
targets = [
    "year_as", "month_as", "day_as", "hour_as", "min_as", "sec_as",
    "lat_as", "lon_as", "depth_as", "class_as"
]

from sklearn.preprocessing import StandardScaler
X = train[features].replace([np.inf, -np.inf], 0).fillna(0)
y = train[targets].replace([np.inf, -np.inf], 0).fillna(0)

scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)

/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [ ]:
from pytorch_tabnet.tab_model import TabNetRegressor
import torch

best_params = {
    "n_d": 16,
    "n_a": 123,
    "n_steps": 3,
    "gamma": 1.515,
    "lr": 0.00984,
    "dropout": 0.00336
}

final_model = TabNetRegressor(
    n_d=best_params["n_d"],
    n_a=best_params["n_a"],
    n_steps=best_params["n_steps"],
    gamma=best_params["gamma"],
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=best_params["lr"]),
    mask_type='entmax',
    n_shared=1,
    n_independent=1,
    seed=42,
    verbose=1
)

final_model.fit(
    X_train=X_scaled, y_train=y_scaled,
    max_epochs=300,
    patience=20,
    batch_size=512,
    virtual_batch_size=128,
    drop_last=False
)

Device used : cpu
No early stopping will be performed, last training weights will be used.
epoch 0  | loss: 3.15505 |  0:00:00s
epoch 1  | loss: 1.60025 |  0:00:00s
epoch 2  | loss: 1.10931 |  0:00:01s
epoch 3  | loss: 0.96301 |  0:00:01s
epoch 4  | loss: 0.90101 |  0:00:02s
epoch 5  | loss: 0.85952 |  0:00:02s
epoch 6  | loss: 0.82268 |  0:00:02s
epoch 7  | loss: 0.79986 |  0:00:03s
epoch 8  | loss: 0.77497 |  0:00:03s
epoch 9  | loss: 0.75567 |  0:00:04s
epoch 10 | loss: 0.73852 |  0:00:04s
epoch 11 | loss: 0.72371 |  0:00:04s
epoch 12 | loss: 0.70923 |  0:00:04s
epoch 13 | loss: 0.69361 |  0:00:05s
epoch 14 | loss: 0.68229 |  0:00:05s
epoch 15 | loss: 0.67809 |  0:00:05s
epoch 16 | loss: 0.66548 |  0:00:05s
epoch 17 | loss: 0.66101 |  0:00:05s
epoch 18 | loss: 0.64926 |  0:00:06s
epoch 19 | loss: 0.64757 |  0:00:06s
epoch 20 | loss: 0.63555 |  0:00:06s
epoch 21 | loss: 0.62803 |  0:00:06s
epoch 22 | loss: 0.62457 |  0:00:06s
epoch 23 | loss: 0.61535 |  0:00:07s
epoch 24 | loss: 0.60

In [ ]:

X_test = test[features].replace([np.inf, -np.inf], 0).fillna(0)
X_test_scaled = scaler_X.transform(X_test)

preds_scaled = final_model.predict(X_test_scaled)
preds = scaler_y.inverse_transform(preds_scaled)


for i, col in enumerate(targets):
    test[col] = preds[:, i]

test["year_as"] = test["year_as"].round().astype(int)
test["month_as"] = test["month_as"].round().astype(int)
test["day_as"] = test["day_as"].round().astype(int)
test["hour_as"] = test["hour_as"].round().astype(int)
test["min_as"] = test["min_as"].round().astype(int)
test["sec_as"] = test["sec_as"].round(3)
test["lat_as"] = test["lat_as"].round(5)
test["lon_as"] = test["lon_as"].round(5)
test["depth_as"] = test["depth_as"].round(2)
test["class_as"] = test["class_as"].round(2)

submission = test[[
    "id_eq", "year_as", "month_as", "day_as", "hour_as", "min_as", "sec_as",
    "lat_as", "lon_as", "depth_as", "class_as"
]]
submission.to_csv("submission_tabnet_optuna.csv", index=False)

from google.colab import files
files.download("submission_tabnet_optuna.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install numpy==1.23.5 --force-reinstall

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 40.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
blosc2 3.2.1 requires numpy>=1.26, but you have numpy 1.23.5 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible.
chex 0.1.89 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatible.
pymc 5.21.2 requires numpy>=1.25.0, but you have numpy 1.23.5 which is incompatible.
bigframes 1.42.0 requires numpy>=1.24.0, but you have numpy 1.23.5 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible.
scikit-image 0.25.2 requires numpy>=1.24, but you have numpy 1.23.5 which is incompati

In [ ]:
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import numpy as np
import pandas as pd
import torch

features = [
    "year", "month", "day", "hour", "min", "sec",
    "lat", "lon", "depth", "class",
    "weekday", "is_weekend", "hour_fraction", "is_night", "season",
    "delta_lat", "delta_lon", "distance_km",
    "log_class", "log_depth", "lat_bin", "lon_bin",
    "lat_times_lon", "depth_div_class"
]
targets = [
    "year_as", "month_as", "day_as", "hour_as", "min_as", "sec_as",
    "lat_as", "lon_as", "depth_as", "class_as"
]

X = train[features].replace([np.inf, -np.inf], 0).fillna(0)
y = train[targets].replace([np.inf, -np.inf], 0).fillna(0)

scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)

X_train, X_val, y_train, y_val = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

model = TabNetRegressor(
    n_d=16, n_a=123, n_steps=3, gamma=1.5,
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=0.009),
    mask_type="sparsemax",
    cat_idxs=[], cat_dims=[], cat_emb_dim=[],
    n_shared=1, n_independent=1,
    momentum=0.3,
    lambda_sparse=1e-3,
    clip_value=2.0,
    verbose=1,
)

model.fit(
    X_train=X_train, y_train=y_train,
    eval_set=[(X_val, y_val)],
    eval_name=["val"],
    eval_metric=["mae"],
    max_epochs=200,
    patience=25,
    batch_size=256,
    virtual_batch_size=128,
    drop_last=False,
)

X_test = test[features].replace([np.inf, -np.inf], 0).fillna(0)
X_test_scaled = scaler_X.transform(X_test)
test_preds_scaled = model.predict(X_test_scaled)
test_preds = scaler_y.inverse_transform(test_preds_scaled)

for i, col in enumerate(targets):
    test[col] = test_preds[:, i]

test["year_as"] = test["year_as"].round().astype(int)
test["month_as"] = test["month_as"].round().astype(int)
test["day_as"] = test["day_as"].round().astype(int)
test["hour_as"] = test["hour_as"].round().astype(int)
test["min_as"] = test["min_as"].round().astype(int)
test["sec_as"] = test["sec_as"].round(3)
test["lat_as"] = test["lat_as"].round(5)
test["lon_as"] = test["lon_as"].round(5)
test["depth_as"] = test["depth_as"].round(2)
test["class_as"] = test["class_as"].round(2)

submission = test[[
    "id_eq", "year_as", "month_as", "day_as", "hour_as", "min_as", "sec_as",
    "lat_as", "lon_as", "depth_as", "class_as"
]]
submission.to_csv("submission_tabnet_best.csv", index=False)
print(" submission_tabnet_best.csv сохранён!")


Device used : cpu
epoch 0  | loss: 3.05839 | val_mae: 1.04776 |  0:00:00s
epoch 1  | loss: 1.6777  | val_mae: 0.92964 |  0:00:00s
epoch 2  | loss: 1.23908 | val_mae: 0.88193 |  0:00:00s
epoch 3  | loss: 1.0732  | val_mae: 0.84835 |  0:00:00s
epoch 4  | loss: 0.98815 | val_mae: 0.82492 |  0:00:01s
epoch 5  | loss: 0.95776 | val_mae: 0.80355 |  0:00:01s
epoch 6  | loss: 0.91167 | val_mae: 0.7967  |  0:00:01s
epoch 7  | loss: 0.88509 | val_mae: 0.79123 |  0:00:01s
epoch 8  | loss: 0.87592 | val_mae: 0.7816  |  0:00:02s
epoch 9  | loss: 0.84248 | val_mae: 0.76856 |  0:00:02s
epoch 10 | loss: 0.82936 | val_mae: 0.76276 |  0:00:02s
epoch 11 | loss: 0.81368 | val_mae: 0.75212 |  0:00:02s
epoch 12 | loss: 0.7997  | val_mae: 0.74512 |  0:00:03s
epoch 13 | loss: 0.78472 | val_mae: 0.74227 |  0:00:03s
epoch 14 | loss: 0.77312 | val_mae: 0.737   |  0:00:03s
epoch 15 | loss: 0.75857 | val_mae: 0.73013 |  0:00:03s
epoch 16 | loss: 0.75139 | val_mae: 0.72581 |  0:00:03s
epoch 17 | loss: 0.74305 | val

In [ ]:
import pandas as pd
import numpy as np
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

def add_features(df):
    df['datetime'] = pd.to_datetime(dict(
        year=df['year'], month=df['month'], day=df['day'],
        hour=df['hour'], minute=df['min'], second=df['sec']
    ), errors='coerce')

    df['weekday'] = df['datetime'].dt.weekday
    df['is_weekend'] = (df['weekday'] >= 5).astype(int)
    df['hour_fraction'] = df['hour'] + df['min'] / 60 + df['sec'] / 3600
    df['is_night'] = ((df['hour'] < 6) | (df['hour'] > 22)).astype(int)
    df['season'] = ((df['month'] % 12 + 3) // 3).astype(int)

    df['delta_lat'] = df['lat_as'] - df['lat'] if 'lat_as' in df else 0
    df['delta_lon'] = df['lon_as'] - df['lon'] if 'lon_as' in df else 0
    df['distance_km'] = np.sqrt(df['delta_lat']**2 + df['delta_lon']**2) * 111

    df['log_class'] = np.log1p(df['class'])
    df['log_depth'] = np.log1p(df['depth'])
    df['lat_bin'] = (df['lat'] * 2).round(0) / 2
    df['lon_bin'] = (df['lon'] * 2).round(0) / 2
    df['lat_times_lon'] = df['lat'] * df['lon']
    df['depth_div_class'] = df['depth'] / (df['class'] + 1e-5)

    df = df.replace([np.inf, -np.inf], 0).fillna(0)
    return df

train = add_features(train)
test = add_features(test)

features = [
    "year", "month", "day", "hour", "min", "sec",
    "lat", "lon", "depth", "class",
    "weekday", "is_weekend", "hour_fraction", "is_night", "season",
    "delta_lat", "delta_lon", "distance_km",
    "log_class", "log_depth", "lat_bin", "lon_bin",
    "lat_times_lon", "depth_div_class"
]
targets = [
    "year_as", "month_as", "day_as", "hour_as", "min_as", "sec_as",
    "lat_as", "lon_as", "depth_as", "class_as"
]

X = train[features].replace([np.inf, -np.inf], 0).fillna(0)
y = train[targets].replace([np.inf, -np.inf], 0).fillna(0)
X_test = test[features].replace([np.inf, -np.inf], 0).fillna(0)

scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)
X_test_scaled = scaler_X.transform(X_test)

model = TabNetRegressor(
    n_d=16, n_a=123, n_steps=3,
    gamma=1.515, lambda_sparse=0.0001,
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=0.009),
    mask_type="entmax",
    scheduler_params={"step_size": 10, "gamma": 0.9},
    scheduler_fn=torch.optim.lr_scheduler.StepLR,
    verbose=0
)

model.fit(
    X_train=X_scaled, y_train=y_scaled,
    eval_set=[(X_scaled, y_scaled)],
    eval_name=["train"],
    eval_metric=["mae"],
    max_epochs=200,
    patience=20,
    batch_size=512,
    virtual_batch_size=128,
    num_workers=0,
    drop_last=False
)

y_pred_scaled = model.predict(X_test_scaled)
y_pred = scaler_y.inverse_transform(y_pred_scaled)

for i, col in enumerate(targets):
    test[col] = y_pred[:, i]

test["year_as"] = test["year_as"].round().astype(int)
test["month_as"] = test["month_as"].round().astype(int)
test["day_as"] = test["day_as"].round().astype(int)
test["hour_as"] = test["hour_as"].round().astype(int)
test["min_as"] = test["min_as"].round().astype(int)
test["sec_as"] = test["sec_as"].round(3)
test["lat_as"] = test["lat_as"].round(5)
test["lon_as"] = test["lon_as"].round(5)
test["depth_as"] = test["depth_as"].round(2)
test["class_as"] = test["class_as"].round(2)

submission = test[[
    "id_eq", "year_as", "month_as", "day_as", "hour_as", "min_as", "sec_as",
    "lat_as", "lon_as", "depth_as", "class_as"
]]
submission.to_csv("submission_tabnet_best.csv", index=False)
print("\n submission_tabnet_best.csv сохранён!")

/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


Stop training because you reached max_epochs = 200 with best_epoch = 199 and best_train_mae = 0.3029
Best weights from best epoch are automatically used!

✅ submission_tabnet_best.csv сохранён!


import optuna
import numpy as np
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
from catboost import CatBoostRegressor

# 📌 Обязательно: X_scaled, y_scaled уже должны быть готовы — без NaN и масштабированы

def objective(trial):
    params = {
        "iterations": trial.suggest_int("iterations", 300, 1000),
        "depth": trial.suggest_int("depth", 4, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.3, log=True),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1.0, 10.0),
        "random_strength": trial.suggest_float("random_strength", 0.5, 2.0),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.0, 1.0),
        "loss_function": "MultiRMSE",
        "verbose": 0,
        "task_type": "GPU",  # ✅ GPU ускорение
        "devices": "0"
    }

    model = MultiOutputRegressor(CatBoostRegressor(**params))

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    maes = []
    for train_idx, val_idx in kf.split(X_scaled):
        X_train, X_val = X_scaled[train_idx], X_scaled[val_idx]
        y_train, y_val = y_scaled[train_idx], y_scaled[val_idx]
        model.fit(X_train, y_train)
        preds = model.predict(X_val)
        mae = mean_absolute_error(y_val, preds)
        maes.append(mae)

    return np.mean(maes)

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=30)

print("✅ Best hyperparameters:", study.best_params)

In [ ]:
!pip install catboost

In [ ]:
!pip install optuna

In [ ]:
!pip install numpy==1.26.4 --quiet

In [ ]:
import optuna
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostRegressor
def add_minimal_features(df):
    df['datetime'] = pd.to_datetime(dict(
        year=df['year'], month=df['month'], day=df['day'],
        hour=df['hour'], minute=df['min'], second=df['sec']
    ), errors='coerce')

    df['day_of_year'] = df['datetime'].dt.dayofyear.fillna(0)
    df['main_eq_time_decimal'] = df['hour'] + df['min'] / 60 + df['sec'] / 3600
    df['cos_hour'] = np.cos(2 * np.pi * df['main_eq_time_decimal'] / 24)
    df['sin_hour'] = np.sin(2 * np.pi * df['main_eq_time_decimal'] / 24)
    return df.replace([np.inf, -np.inf], np.nan).fillna(0)

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
train = add_minimal_features(train)
test = add_minimal_features(test)

features = [
    "year", "month", "day", "hour", "min", "sec",
    "lat", "lon", "depth", "class",
    "day_of_year", "cos_hour", "sin_hour"
]
targets = [
    "year_as", "month_as", "day_as", "hour_as", "min_as", "sec_as",
    "lat_as", "lon_as", "depth_as", "class_as"
]

X = train[features].replace([np.inf, -np.inf], 0).fillna(0)
y = train[targets].replace([np.inf, -np.inf], 0).fillna(0)

scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)

def objective(trial):
    model = MultiOutputRegressor(CatBoostRegressor(
        iterations=trial.suggest_int("iterations", 300, 1000),
        depth=trial.suggest_int("depth", 4, 10),
        learning_rate=trial.suggest_float("learning_rate", 0.005, 0.05),
        l2_leaf_reg=trial.suggest_float("l2_leaf_reg", 1.0, 10.0),
        random_strength=trial.suggest_float("random_strength", 0.5, 2.0),
        bagging_temperature=trial.suggest_float("bagging_temperature", 0, 1),
        verbose=0,
        task_type="GPU",
        boosting_type="Plain",
        loss_function="RMSE"
    ))

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    maes = []
    for train_idx, val_idx in kf.split(X_scaled):
        X_train, X_val = X_scaled[train_idx], X_scaled[val_idx]
        y_train, y_val = y_scaled[train_idx], y_scaled[val_idx]

        model.fit(X_train, y_train)
        preds = model.predict(X_val)
        maes.append(mean_absolute_error(y_val, preds))

    return np.mean(maes)

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=30)
print("\n Best hyperparameters:", study.best_params)

best_params = study.best_params
final_model = MultiOutputRegressor(CatBoostRegressor(
    **best_params,
    verbose=0,
    task_type="GPU",
    boosting_type="Plain",
    loss_function="RMSE"
))
final_model.fit(X_scaled, y_scaled)

X_test = test[features].replace([np.inf, -np.inf], 0).fillna(0)
X_test_scaled = scaler_X.transform(X_test)
preds_scaled = final_model.predict(X_test_scaled)
preds = scaler_y.inverse_transform(preds_scaled)

for i, col in enumerate(targets):
    test[col] = preds[:, i]

test["year_as"] = test["year_as"].round().astype(int)
test["month_as"] = test["month_as"].round().astype(int)
test["day_as"] = test["day_as"].round().astype(int)
test["hour_as"] = test["hour_as"].round().astype(int)
test["min_as"] = test["min_as"].round().astype(int)
test["sec_as"] = test["sec_as"].round(3)
test["lat_as"] = test["lat_as"].round(5)
test["lon_as"] = test["lon_as"].round(5)
test["depth_as"] = test["depth_as"].round(2)
test["class_as"] = test["class_as"].round(2)

submission = test[[
    "id_eq", "year_as", "month_as", "day_as", "hour_as", "min_as", "sec_as",
    "lat_as", "lon_as", "depth_as", "class_as"
]]
submission.to_csv("submission_catboost_optuna.csv", index=False)
print("\n submission_catboost_optuna.csv сохранён!")


[I 2025-04-13 05:26:57,443] A new study created in memory with name: no-name-1beacb9b-9d2f-470f-bd3d-94f249eae69f
[I 2025-04-13 05:28:41,710] Trial 0 finished with value: 0.432386653932243 and parameters: {'iterations': 443, 'depth': 7, 'learning_rate': 0.011904643436961009, 'l2_leaf_reg': 6.184467391262881, 'random_strength': 0.9503521296686035, 'bagging_temperature': 0.2552529941781061}. Best is trial 0 with value: 0.432386653932243.
[I 2025-04-13 05:32:18,088] Trial 1 finished with value: 0.43450013723257086 and parameters: {'iterations': 887, 'depth': 8, 'learning_rate': 0.005673676104262354, 'l2_leaf_reg': 3.447668594338533, 'random_strength': 1.522961206527982, 'bagging_temperature': 0.21780071044552451}. Best is trial 0 with value: 0.432386653932243.
[I 2025-04-13 05:34:27,908] Trial 2 finished with value: 0.42581720872156853 and parameters: {'iterations': 836, 'depth': 4, 'learning_rate': 0.020378168903327935, 'l2_leaf_reg': 4.506854896507322, 'random_strength': 0.5708415943128

KeyboardInterrupt: 

In [ ]:
import optuna
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostRegressor

def add_minimal_features(df):
    df['datetime'] = pd.to_datetime(dict(
        year=df['year'], month=df['month'], day=df['day'],
        hour=df['hour'], minute=df['min'], second=df['sec']
    ), errors='coerce')

    df['day_of_year'] = df['datetime'].dt.dayofyear.fillna(0)
    df['main_eq_time_decimal'] = df['hour'] + df['min'] / 60 + df['sec'] / 3600
    df['cos_hour'] = np.cos(2 * np.pi * df['main_eq_time_decimal'] / 24)
    df['sin_hour'] = np.sin(2 * np.pi * df['main_eq_time_decimal'] / 24)

    df = df.replace([np.inf, -np.inf], np.nan).fillna(0)
    return df

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
train = add_minimal_features(train)
test = add_minimal_features(test)
def add_full_features(df):
    df['datetime'] = pd.to_datetime(dict(
        year=df['year'], month=df['month'], day=df['day'],
        hour=df['hour'], minute=df['min'], second=df['sec']
    ), errors='coerce')

    df["weekday"] = df['datetime'].dt.weekday.fillna(0).astype(int)
    df["is_weekend"] = df["weekday"].isin([5, 6]).astype(int)
    df["hour_fraction"] = df["hour"] + df["min"] / 60 + df["sec"] / 3600
    df["is_night"] = df["hour"].between(0, 6).astype(int)
    df["season"] = ((df["month"] % 12 + 3) // 3).astype(int)

    df["delta_lat"] = df["lat_as"] - df["lat"]
    df["delta_lon"] = df["lon_as"] - df["lon"]
    df["distance_km"] = np.sqrt(df["delta_lat"]**2 + df["delta_lon"]**2) * 111

    df["log_class"] = np.log1p(df["class"])
    df["log_depth"] = np.log1p(df["depth"])
    df["lat_bin"] = pd.cut(df["lat"], bins=10, labels=False)
    df["lon_bin"] = pd.cut(df["lon"], bins=10, labels=False)
    df["lat_times_lon"] = df["lat"] * df["lon"]
    df["depth_div_class"] = df["depth"] / (df["class"] + 1)

    return df.fillna(0)

train = add_full_features(train)
test = add_full_features(test)

features = [
    "year", "month", "day", "hour", "min", "sec",
    "lat", "lon", "depth", "class",
    "weekday", "is_weekend", "hour_fraction", "is_night", "season",
    "delta_lat", "delta_lon", "distance_km",
    "log_class", "log_depth", "lat_bin", "lon_bin",
    "lat_times_lon", "depth_div_class",
    "day_of_year", "cos_hour", "sin_hour"
]
targets = [
    "year_as", "month_as", "day_as", "hour_as", "min_as", "sec_as",
    "lat_as", "lon_as", "depth_as", "class_as"
]

X = train[features].replace([np.inf, -np.inf], 0).fillna(0)
y = train[targets].replace([np.inf, -np.inf], 0).fillna(0)

scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)

def objective(trial):
    model = MultiOutputRegressor(CatBoostRegressor(
        iterations=trial.suggest_int("iterations", 300, 1000),
        depth=trial.suggest_int("depth", 4, 10),
        learning_rate=trial.suggest_float("learning_rate", 0.005, 0.05),
        l2_leaf_reg=trial.suggest_float("l2_leaf_reg", 1.0, 10.0),
        random_strength=trial.suggest_float("random_strength", 0.5, 2.0),
        bagging_temperature=trial.suggest_float("bagging_temperature", 0, 1),
        verbose=0,
        task_type="GPU",
        boosting_type="Plain"
    ))

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    maes = []
    for train_idx, val_idx in kf.split(X_scaled):
        X_train, X_val = X_scaled[train_idx], X_scaled[val_idx]
        y_train, y_val = y_scaled[train_idx], y_scaled[val_idx]
        model.fit(X_train, y_train)
        preds = model.predict(X_val)
        maes.append(mean_absolute_error(y_val, preds))

    return np.mean(maes)


study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=30)

print("\n Best hyperparameters:", study.best_params)

/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


KeyError: 'lat_as'

In [ ]:
import optuna
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostRegressor

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

def add_full_features(df, is_train=True):
    df['datetime'] = pd.to_datetime(dict(
        year=df['year'], month=df['month'], day=df['day'],
        hour=df['hour'], minute=df['min'], second=df['sec']
    ), errors='coerce')

    df["weekday"] = df['datetime'].dt.weekday.fillna(0).astype(int)
    df["is_weekend"] = df["weekday"].isin([5, 6]).astype(int)
    df["hour_fraction"] = df["hour"] + df["min"] / 60 + df["sec"] / 3600
    df["is_night"] = df["hour"].between(0, 6).astype(int)
    df["season"] = ((df["month"] % 12 + 3) // 3).astype(int)

    df["log_class"] = np.log1p(df["class"])
    df["log_depth"] = np.log1p(df["depth"])
    df["lat_bin"] = pd.cut(df["lat"], bins=10, labels=False)
    df["lon_bin"] = pd.cut(df["lon"], bins=10, labels=False)
    df["lat_times_lon"] = df["lat"] * df["lon"]
    df["depth_div_class"] = df["depth"] / (df["class"] + 1)

    if is_train:
        df["delta_lat"] = df["lat_as"] - df["lat"]
        df["delta_lon"] = df["lon_as"] - df["lon"]
        df["distance_km"] = np.sqrt(df["delta_lat"]**2 + df["delta_lon"]**2) * 111
    else:
        df["delta_lat"] = 0
        df["delta_lon"] = 0
        df["distance_km"] = 0

    return df.fillna(0)

train = add_full_features(train, is_train=True)
test = add_full_features(test, is_train=False)

features = [
    "year", "month", "day", "hour", "min", "sec",
    "lat", "lon", "depth", "class",
    "weekday", "is_weekend", "hour_fraction", "is_night", "season",
    "delta_lat", "delta_lon", "distance_km",
    "log_class", "log_depth", "lat_bin", "lon_bin",
    "lat_times_lon", "depth_div_class"
]
targets = [
    "year_as", "month_as", "day_as", "hour_as", "min_as", "sec_as",
    "lat_as", "lon_as", "depth_as", "class_as"
]

X = train[features].replace([np.inf, -np.inf], 0).fillna(0)
y = train[targets].replace([np.inf, -np.inf], 0).fillna(0)

scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)

def objective(trial):
    model = MultiOutputRegressor(CatBoostRegressor(
        iterations=trial.suggest_int("iterations", 300, 1000),
        depth=trial.suggest_int("depth", 4, 10),
        learning_rate=trial.suggest_float("learning_rate", 0.005, 0.05),
        l2_leaf_reg=trial.suggest_float("l2_leaf_reg", 1.0, 10.0),
        random_strength=trial.suggest_float("random_strength", 0.5, 2.0),
        bagging_temperature=trial.suggest_float("bagging_temperature", 0, 1),
        verbose=0,
        task_type="GPU",
        boosting_type="Plain"
    ))

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    maes = []

    for train_idx, val_idx in kf.split(X_scaled):
        X_train, X_val = X_scaled[train_idx], X_scaled[val_idx]
        y_train, y_val = y_scaled[train_idx], y_scaled[val_idx]

        model.fit(X_train, y_train)
        preds = model.predict(X_val)
        maes.append(mean_absolute_error(y_val, preds))

    return np.mean(maes)

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=30)

print("\n Best hyperparameters:", study.best_params)

/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
[I 2025-04-13 05:42:53,007] A new study created in memory with name: no-name-ea604df6-15fb-4736-a41d-05a49a679e43
[W 2025-04-13 05:46:51,548] Trial 0 failed with parameters: {'iterations': 945, 'depth': 10, 'learning_rate': 0.043224052637879165, 'l2_leaf_reg': 5.281869161046789, 'random_strength': 0.8852351886029288, 'bagging_temperature': 0.18551600335231588} because of the following error: KeyboardInterrupt('').
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "<ipython-input-10-8f6ead4572f4>", line 91, in objective
    model.fit(X_train, y_train)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1389, in wrapper
    return fit_metho

KeyboardInterrupt: 

In [ ]:

import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_absolute_error
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostRegressor
from pytorch_tabnet.tab_model import TabNetRegressor
import torch
import torch.nn as nn

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

def add_features(df):
    df['datetime'] = pd.to_datetime(dict(year=df['year'], month=df['month'], day=df['day'],
                                         hour=df['hour'], minute=df['min'], second=df['sec']), errors='coerce')
    df['day_of_year'] = df['datetime'].dt.dayofyear.fillna(0)
    df['main_eq_time_decimal'] = df['hour'] + df['min'] / 60 + df['sec'] / 3600
    df['cos_hour'] = np.cos(2 * np.pi * df['main_eq_time_decimal'] / 24)
    df['sin_hour'] = np.sin(2 * np.pi * df['main_eq_time_decimal'] / 24)
    df = df.replace([np.inf, -np.inf], np.nan).fillna(0)
    return df

train = add_features(train)
test = add_features(test)

features = [
    "year", "month", "day", "hour", "min", "sec", "lat", "lon", "depth", "class",
    "day_of_year", "main_eq_time_decimal", "cos_hour", "sin_hour"
]
targets = [
    "year_as", "month_as", "day_as", "hour_as", "min_as", "sec_as",
    "lat_as", "lon_as", "depth_as", "class_as"
]

X = train[features].copy()
y = train[targets].copy()
X_test = test[features].copy()

scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)
X_test_scaled = scaler_X.transform(X_test)

class PyTorchModel(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 128), nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(128, 64), nn.ReLU(),
            nn.Linear(64, output_size)
        )

    def forward(self, x):
        return self.model(x)

oof_preds_tabnet = np.zeros_like(y_scaled)
oof_preds_cat = np.zeros_like(y_scaled)
oof_preds_pytorch = np.zeros_like(y_scaled)

kf = KFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(kf.split(X_scaled)):
    print(f"Fold {fold+1}")
    X_train, X_val = X_scaled[train_idx], X_scaled[val_idx]
    y_train, y_val = y_scaled[train_idx], y_scaled[val_idx]

    # TabNet
    tabnet = TabNetRegressor(n_d=16, n_a=123, n_steps=3, gamma=1.5, lambda_sparse=1e-4,
                              optimizer_fn=torch.optim.Adam, optimizer_params=dict(lr=0.009),
                              verbose=0)
    tabnet.fit(X_train, y_train, eval_set=[(X_val, y_val)], patience=30)
    oof_preds_tabnet[val_idx] = tabnet.predict(X_val)

    # CatBoost
    cat = MultiOutputRegressor(CatBoostRegressor(verbose=0, task_type='GPU', iterations=500))
    cat.fit(X_train, y_train)
    oof_preds_cat[val_idx] = cat.predict(X_val)

    # PyTorch
    model = PyTorchModel(input_size=X_train.shape[1], output_size=y_train.shape[1])
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    loss_fn = nn.MSELoss()

    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
    X_val_tensor = torch.tensor(X_val, dtype=torch.float32)

    for epoch in range(50):
        model.train()
        optimizer.zero_grad()
        loss = loss_fn(model(X_train_tensor), y_train_tensor)
        loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        preds = model(X_val_tensor).numpy()
        oof_preds_pytorch[val_idx] = preds


X_meta = np.hstack([oof_preds_tabnet, oof_preds_cat, oof_preds_pytorch])
meta_model = MultiOutputRegressor(RidgeCV())
meta_model.fit(X_meta, y_scaled)

tabnet.fit(X_scaled, y_scaled)
preds_tabnet = tabnet.predict(X_test_scaled)

cat.fit(X_scaled, y_scaled)
preds_cat = cat.predict(X_test_scaled)

X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y_scaled, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)

model = PyTorchModel(input_size=X_scaled.shape[1], output_size=y_scaled.shape[1])
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()
for epoch in range(50):
    model.train()
    optimizer.zero_grad()
    loss = loss_fn(model(X_tensor), y_tensor)
    loss.backward()
    optimizer.step()

model.eval()
with torch.no_grad():
    preds_pytorch = model(X_test_tensor).numpy()

X_test_meta = np.hstack([preds_tabnet, preds_cat, preds_pytorch])
final_preds_scaled = meta_model.predict(X_test_meta)
final_preds = scaler_y.inverse_transform(final_preds_scaled)

for i, col in enumerate(targets):
    test[col] = final_preds[:, i]

test["year_as"] = test["year_as"].round().astype(int)
test["month_as"] = test["month_as"].round().astype(int)
test["day_as"] = test["day_as"].round().astype(int)
test["hour_as"] = test["hour_as"].round().astype(int)
test["min_as"] = test["min_as"].round().astype(int)
test["sec_as"] = test["sec_as"].round(3)
test["lat_as"] = test["lat_as"].round(5)
test["lon_as"] = test["lon_as"].round(5)
test["depth_as"] = test["depth_as"].round(2)
test["class_as"] = test["class_as"].round(2)

submission = test[["id_eq"] + targets]
submission.to_csv("submission_stacked.csv", index=False)
print("\n submission_stacked.csv сохранён!")

Fold 1

Early stopping occurred at epoch 30 with best_epoch = 0 and best_val_0_mse = 143.433


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Fold 2

Early stopping occurred at epoch 30 with best_epoch = 0 and best_val_0_mse = 109.80448


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Fold 3

Early stopping occurred at epoch 30 with best_epoch = 0 and best_val_0_mse = 151.03052


...

Fold 4

Early stopping occurred at epoch 30 with best_epoch = 0 and best_val_0_mse = 121.88463


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Fold 5

Early stopping occurred at epoch 30 with best_epoch = 0 and best_val_0_mse = 193.13645


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)



✅ submission_stacked.csv сохранён!


In [ ]:
pip install pytorch-tabnet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [ ]:
pip uninstall torch
pip install torch --force-reinstall --no-cache-dir

SyntaxError: invalid syntax (<ipython-input-15-4de11f26c60b>, line 1)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import RidgeCV
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostRegressor
from pytorch_tabnet.tab_model import TabNetRegressor
import torch


train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

def feature_engineering(df):
    df['datetime'] = pd.to_datetime(dict(year=df['year'], month=df['month'], day=df['day'],
                                         hour=df['hour'], minute=df['min'], second=df['sec']), errors='coerce')
    df['hour_fraction'] = df['hour'] + df['min'] / 60 + df['sec'] / 3600
    df['weekday'] = df['datetime'].dt.weekday.fillna(0)
    df['is_weekend'] = (df['weekday'] >= 5).astype(int)
    df['is_night'] = ((df['hour'] < 6) | (df['hour'] > 22)).astype(int)
    df['season'] = ((df['month'] % 12 + 3) // 3)

    df['delta_lat'] = df['lat'].diff().fillna(0)
    df['delta_lon'] = df['lon'].diff().fillna(0)
    df['distance_km'] = np.sqrt(df['delta_lat']**2 + df['delta_lon']**2) * 111

    df['log_class'] = np.log1p(df['class'].clip(lower=0))
    df['log_depth'] = np.log1p(df['depth'].clip(lower=0))

    df['lat_bin'] = pd.qcut(df['lat'], q=10, labels=False, duplicates='drop')
    df['lon_bin'] = pd.qcut(df['lon'], q=10, labels=False, duplicates='drop')
    df['lat_times_lon'] = df['lat'] * df['lon']
    df['depth_div_class'] = df['depth'] / (df['class'] + 1e-5)

    return df.fillna(0)

train = feature_engineering(train)
test = feature_engineering(test)

features = [
    "year", "month", "day", "hour", "min", "sec", "lat", "lon", "depth", "class",
    "weekday", "is_weekend", "hour_fraction", "is_night", "season",
    "delta_lat", "delta_lon", "distance_km", "log_class", "log_depth",
    "lat_bin", "lon_bin", "lat_times_lon", "depth_div_class"
]

targets = [
    "year_as", "month_as", "day_as", "hour_as", "min_as", "sec_as",
    "lat_as", "lon_as", "depth_as", "class_as"
]

X = train[features].replace([np.inf, -np.inf], 0).fillna(0)
y = train[targets].replace([np.inf, -np.inf], 0).fillna(0)
X_test = test[features].replace([np.inf, -np.inf], 0).fillna(0)

scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)
X_test_scaled = scaler_X.transform(X_test)

tabnet_oof = np.zeros_like(y_scaled)
catboost_oof = np.zeros_like(y_scaled)

tabnet_test_preds = []
catboost_test_preds = []

kf = KFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(kf.split(X_scaled)):
    print(f"\nFold {fold+1}")

    X_tr, X_val = X_scaled[train_idx], X_scaled[val_idx]
    y_tr, y_val = y_scaled[train_idx], y_scaled[val_idx]

    # === TabNet ===
    tabnet = TabNetRegressor(
        n_d=16, n_a=123, n_steps=3, gamma=1.5,
        optimizer_fn=torch.optim.Adam,
        optimizer_params=dict(lr=0.009),
        verbose=0,
        seed=42
    )

    tabnet.fit(X_tr, y_tr, eval_set=[(X_val, y_val)], patience=30)
    tabnet_oof[val_idx] = tabnet.predict(X_val)
    tabnet_test_preds.append(tabnet.predict(X_test_scaled))

    # === CatBoost ===
    cat = MultiOutputRegressor(CatBoostRegressor(
        iterations=600,
        depth=8,
        learning_rate=0.015,
        l2_leaf_reg=4,
        random_strength=1,
        bagging_temperature=0.3,
        verbose=0,
        task_type="CPU"
    ))
    cat.fit(X_tr, y_tr)
    catboost_oof[val_idx] = cat.predict(X_val)
    catboost_test_preds.append(cat.predict(X_test_scaled))

X_stack_train = np.hstack([tabnet_oof, catboost_oof])
X_stack_test = np.hstack([np.mean(tabnet_test_preds, axis=0),
                          np.mean(catboost_test_preds, axis=0)])

stack_model = MultiOutputRegressor(RidgeCV())
stack_model.fit(X_stack_train, y_scaled)
y_pred_scaled = stack_model.predict(X_stack_test)
y_pred = scaler_y.inverse_transform(y_pred_scaled)

submission = pd.DataFrame(y_pred, columns=targets)
submission["id_eq"] = test["id_eq"]
submission = submission[["id_eq"] + targets]
submission.to_csv("submission_stacked.csv", index=False)

print("\n✅ submission_stacked.csv сохранён!")



Fold 1

Early stopping occurred at epoch 30 with best_epoch = 0 and best_val_0_mse = 176.01292


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



Fold 2

Early stopping occurred at epoch 30 with best_epoch = 0 and best_val_0_mse = 118.05576


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
/usr/local/lib/python3.11/dist-packages/torch/utils/data/_utils/collate.py:285: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at /pytorch/torch/csrc/utils/tensor_numpy.cpp:203.)
  return collate([torch.as_tensor(b) for b in batch], collate_fn_map=collate_fn_map)



Fold 3

Early stopping occurred at epoch 30 with best_epoch = 0 and best_val_0_mse = 144.80863


...


Fold 4

Early stopping occurred at epoch 30 with best_epoch = 0 and best_val_0_mse = 137.6973


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



Fold 5

Early stopping occurred at epoch 30 with best_epoch = 0 and best_val_0_mse = 148.35029


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



✅ submission_stacked.csv сохранён!


In [ ]:
import pandas as pd
import numpy as np
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error


train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")


features = [
    "year", "month", "day", "hour", "min", "sec",
    "lat", "lon", "depth", "class",
    "weekday", "is_weekend", "hour_fraction", "is_night", "season",
    "delta_lat", "delta_lon", "distance_km",
    "log_class", "log_depth", "lat_bin", "lon_bin",
    "lat_times_lon", "depth_div_class",
    "day_of_year", "cos_hour", "sin_hour"
]

targets = [
    "year_as", "month_as", "day_as", "hour_as", "min_as", "sec_as",
    "lat_as", "lon_as", "depth_as", "class_as"
]


def add_minimal_features(df):
    df['datetime'] = pd.to_datetime(dict(
        year=df['year'], month=df['month'], day=df['day'],
        hour=df['hour'], minute=df['min'], second=df['sec']
    ), errors='coerce')

    df['day_of_year'] = df['datetime'].dt.dayofyear.fillna(0)
    df['main_eq_time_decimal'] = df['hour'] + df['min'] / 60 + df['sec'] / 3600
    df['cos_hour'] = np.cos(2 * np.pi * df['main_eq_time_decimal'] / 24)
    df['sin_hour'] = np.sin(2 * np.pi * df['main_eq_time_decimal'] / 24)
    df = df.replace([np.inf, -np.inf], np.nan).fillna(0)
    return df

def add_engineered_features(df):
    df["weekday"] = df["datetime"].dt.weekday
    df["is_weekend"] = (df["weekday"] >= 5).astype(int)
    df["hour_fraction"] = df["hour"] + df["min"] / 60
    df["is_night"] = ((df["hour"] < 6) | (df["hour"] >= 22)).astype(int)
    df["season"] = ((df["month"] % 12 + 3) // 3).astype(int)

    df["delta_lat"] = df["lat"].diff().fillna(0)
    df["delta_lon"] = df["lon"].diff().fillna(0)
    df["distance_km"] = np.sqrt(df["delta_lat"]**2 + df["delta_lon"]**2) * 111

    df["log_class"] = np.log1p(df["class"].clip(lower=0))
    df["log_depth"] = np.log1p(df["depth"].clip(lower=0))

    df["lat_bin"] = pd.qcut(df["lat"], q=10, duplicates='drop').cat.codes
    df["lon_bin"] = pd.qcut(df["lon"], q=10, duplicates='drop').cat.codes

    df["lat_times_lon"] = df["lat"] * df["lon"]
    df["depth_div_class"] = df["depth"] / (df["class"] + 1)
    return df

train = add_minimal_features(train)
test = add_minimal_features(test)
train = add_engineered_features(train)
test = add_engineered_features(test)

X = train[features].replace([np.inf, -np.inf], 0).fillna(0)
y = train[targets].replace([np.inf, -np.inf], 0).fillna(0)
X_test = test[features].replace([np.inf, -np.inf], 0).fillna(0)

scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)
X_test_scaled = scaler_X.transform(X_test)

models = []
for i in range(y_scaled.shape[1]):
    model_i = TabNetRegressor(
        n_d=16, n_a=123, n_steps=3, gamma=1.515,
        optimizer_params=dict(lr=0.009),
        scheduler_params={"step_size": 20, "gamma": 0.9},
        scheduler_fn=torch.optim.lr_scheduler.StepLR,
        mask_type='entmax', verbose=0
    )
    model_i.fit(X_scaled, y_scaled[:, [i]])
    models.append(model_i)


preds_scaled = np.column_stack([model.predict(X_test_scaled) for model in models])
preds = scaler_y.inverse_transform(preds_scaled)

for i, col in enumerate(targets):
    test[col] = preds[:, i]

test["year_as"] = test["year_as"].round().astype(int)
test["month_as"] = test["month_as"].round().astype(int)
test["day_as"] = test["day_as"].round().astype(int)
test["hour_as"] = test["hour_as"].round().astype(int)
test["min_as"] = test["min_as"].round().astype(int)
test["sec_as"] = test["sec_as"].round(3)
test["lat_as"] = test["lat_as"].round(5)
test["lon_as"] = test["lon_as"].round(5)
test["depth_as"] = test["depth_as"].round(2)
test["class_as"] = test["class_as"].round(2)

submission = test[[
    "id_eq", "year_as", "month_as", "day_as", "hour_as", "min_as", "sec_as",
    "lat_as", "lon_as", "depth_as", "class_as"
]]
submission.to_csv("submission_tabnet_best.csv", index=False)
print(" submission_tabnet_best.csv сохранён!")

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)
/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)
/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)
/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)
/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)
/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/

✅ submission_tabnet_best.csv сохранён!


In [ ]:
print(train.columns)
print(targets)
missing_cols = [col for col in targets if col not in train.columns]
print(" Missing target columns:", missing_cols)

Index(['Unnamed: 0', 'year', 'month', 'day', 'hour', 'min', 'sec', 'lat',
       'lon', 'depth', 'class', 'year_as', 'month_as', 'day_as', 'hour_as',
       'min_as', 'sec_as', 'lat_as', 'lon_as', 'depth_as', 'class_as',
       'datetime', 'day_of_year', 'main_eq_time_decimal', 'cos_hour',
       'sin_hour', 'weekday', 'is_weekend', 'hour_fraction', 'is_night',
       'season', 'delta_lat', 'delta_lon', 'distance_km', 'log_class',
       'log_depth', 'lat_bin', 'lon_bin', 'lat_times_lon', 'depth_div_class'],
      dtype='object')
['year_as', 'month_as', 'day_as', 'hour_as', 'min_as', 'sec_as', 'lat_as', 'lon_as', 'depth_as', 'class_as']
❌ Missing target columns: []


In [ ]:
print("y.shape:", y.shape)
print("y_scaled.shape:", y_scaled.shape)

y.shape: (1250, 10)
y_scaled.shape: (1250, 10)


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.linear_model import RidgeCV
from sklearn.preprocessing import StandardScaler
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_absolute_error
from catboost import CatBoostRegressor
from pytorch_tabnet.tab_model import TabNetRegressor
import torch
import torch.nn as nn
from datetime import timedelta

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")


def add_features(df):
    df['datetime'] = pd.to_datetime(dict(year=df['year'], month=df['month'], day=df['day'],
                                         hour=df['hour'], minute=df['min'], second=df['sec']), errors='coerce')
    df['day_of_year'] = df['datetime'].dt.dayofyear.fillna(0)
    df['hour_fraction'] = df['hour'] + df['min'] / 60 + df['sec'] / 3600
    df['cos_hour'] = np.cos(2 * np.pi * df['hour_fraction'] / 24)
    df['sin_hour'] = np.sin(2 * np.pi * df['hour_fraction'] / 24)
    df['is_night'] = df['hour'].apply(lambda x: 1 if x <= 6 or x >= 22 else 0)
    df['weekday'] = df['datetime'].dt.weekday.fillna(0)
    df['is_weekend'] = df['weekday'] >= 5
    df['season'] = ((df['month'] % 12 + 3) // 3)
    df['distance_from_equator'] = np.abs(df['lat'])
    df['distance_from_prime_meridian'] = np.abs(df['lon'])
    return df.replace([np.inf, -np.inf], np.nan).fillna(0)

train = add_features(train)
test = add_features(test)

features = [
    "year", "month", "day", "hour", "min", "sec", "lat", "lon", "depth", "class",
    "day_of_year", "hour_fraction", "cos_hour", "sin_hour", "is_night",
    "weekday", "is_weekend", "season", "distance_from_equator", "distance_from_prime_meridian"
]


train['main_dt'] = pd.to_datetime(train[['year', 'month', 'day', 'hour', 'min', 'sec']].astype(str)
                                    .apply(lambda x: '-'.join(x.str.zfill(2)), axis=1))

train['aftershock_dt'] = pd.to_datetime(train[['year_as', 'month_as', 'day_as', 'hour_as', 'min_as']].astype(str)
                                         .apply(lambda x: '-'.join(x.str.zfill(2)), axis=1)) + pd.to_timedelta(train['sec_as'], unit='s')

train['delta_seconds'] = (train['aftershock_dt'] - train['main_dt']).dt.total_seconds()

X = train[features].copy()
y = train['delta_seconds'].copy()
X_test = test[features].copy()

scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1)).ravel()
X_test_scaled = scaler_X.transform(X_test)

class PyTorchModel(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 128), nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(128, 64), nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        return self.model(x)


oof_tabnet = np.zeros_like(y_scaled)
oof_cat = np.zeros_like(y_scaled)
oof_pytorch = np.zeros_like(y_scaled)

kf = KFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(kf.split(X_scaled)):
    print(f"Fold {fold+1}")
    X_tr, X_val = X_scaled[train_idx], X_scaled[val_idx]
    y_tr, y_val = y_scaled[train_idx], y_scaled[val_idx]

    # TabNet
    tabnet = TabNetRegressor(n_d=16, n_a=123, n_steps=3, gamma=1.5, lambda_sparse=1e-4,
                              optimizer_fn=torch.optim.Adam, optimizer_params=dict(lr=0.009), verbose=0)
    tabnet.fit(X_tr, y_tr.reshape(-1,1), eval_set=[(X_val, y_val.reshape(-1,1))], patience=30)
    oof_tabnet[val_idx] = tabnet.predict(X_val).ravel()

    # CatBoost
    cat = CatBoostRegressor(verbose=0, task_type='GPU', iterations=500)
    cat.fit(X_tr, y_tr)
    oof_cat[val_idx] = cat.predict(X_val)

    # PyTorch
    model = PyTorchModel(input_size=X_tr.shape[1])
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    loss_fn = nn.MSELoss()
    X_tr_tensor = torch.tensor(X_tr, dtype=torch.float32)
    y_tr_tensor = torch.tensor(y_tr.reshape(-1,1), dtype=torch.float32)
    X_val_tensor = torch.tensor(X_val, dtype=torch.float32)

    for epoch in range(50):
        model.train()
        optimizer.zero_grad()
        loss = loss_fn(model(X_tr_tensor), y_tr_tensor)
        loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        oof_pytorch[val_idx] = model(X_val_tensor).numpy().ravel()

X_meta = np.vstack([oof_tabnet, oof_cat, oof_pytorch]).T
meta = RidgeCV()
meta.fit(X_meta, y_scaled)

tabnet.fit(X_scaled, y_scaled.reshape(-1,1))
pred_tabnet = tabnet.predict(X_test_scaled).ravel()

cat.fit(X_scaled, y_scaled)
pred_cat = cat.predict(X_test_scaled)

model = PyTorchModel(input_size=X_scaled.shape[1])
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y_scaled.reshape(-1,1), dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)

for epoch in range(50):
    model.train()
    optimizer.zero_grad()
    loss = loss_fn(model(X_tensor), y_tensor)
    loss.backward()
    optimizer.step()

model.eval()
with torch.no_grad():
    pred_pytorch = model(X_test_tensor).numpy().ravel()

X_test_meta = np.vstack([pred_tabnet, pred_cat, pred_pytorch]).T
final_delta_scaled = meta.predict(X_test_meta)
final_delta_seconds = scaler_y.inverse_transform(final_delta_scaled.reshape(-1,1)).ravel()


test['main_dt'] = pd.to_datetime(test[['year', 'month', 'day', 'hour', 'min', 'sec']])
test['aftershock_dt'] = test['main_dt'] + pd.to_timedelta(final_delta_seconds, unit='s')

test["year_as"] = test["aftershock_dt"].dt.year
test["month_as"] = test["aftershock_dt"].dt.month
test["day_as"] = test["aftershock_dt"].dt.day
test["hour_as"] = test["aftershock_dt"].dt.hour
test["min_as"] = test["aftershock_dt"].dt.minute
test["sec_as"] = test["aftershock_dt"].dt.second + test["aftershock_dt"].dt.microsecond / 1e6

target_other = ["lat_as", "lon_as", "depth_as", "class_as"]
multi_cat = MultiOutputRegressor(CatBoostRegressor(verbose=0, iterations=500, task_type='GPU'))
multi_cat.fit(X, train[target_other])
pred_others = multi_cat.predict(X_test)

for i, col in enumerate(target_other):
    test[col] = pred_others[:, i]

test["year_as"] = test["year_as"].astype(int)
test["month_as"] = test["month_as"].astype(int)
test["day_as"] = test["day_as"].astype(int)
test["hour_as"] = test["hour_as"].astype(int)
test["min_as"] = test["min_as"].astype(int)
test["sec_as"] = test["sec_as"].round(3)
test["lat_as"] = test["lat_as"].round(5)
test["lon_as"] = test["lon_as"].round(5)
test["depth_as"] = test["depth_as"].round(2)
test["class_as"] = test["class_as"].round(2)

submission = test[["id_eq", "year_as", "month_as", "day_as", "hour_as", "min_as", "sec_as",
                   "lat_as", "lon_as", "depth_as", "class_as"]]
submission.to_csv("submission_final_stack.csv", index=False)
print(" submission_final_stack.csv сохранён!")


<ipython-input-25-bee37f686729>:46: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  train['main_dt'] = pd.to_datetime(train[['year', 'month', 'day', 'hour', 'min', 'sec']].astype(str)


DateParseError: Unknown datetime string format, unable to parse: 1980-02-15-09-09-350, at position 0

In [ ]:

train['main_dt'] = pd.to_datetime(train[['year', 'month', 'day', 'hour', 'min', 'sec']].astype(str)
                                    .apply(lambda x: '-'.join(x.str.zfill(2)), axis=1) + '-' +
                                    train['sec'].apply(lambda x: str(x).split('.')[0]).str.zfill(2))


train['sec_as_ms'] = train['sec'].apply(lambda x: str(x).split('.')[1] if '.' in str(x) else '000')

train['main_dt_ms'] = pd.to_datetime(train['main_dt'].astype(str) + '.' + train['sec_as_ms'])

train['aftershock_dt'] = pd.to_datetime(train[['year_as', 'month_as', 'day_as', 'hour_as', 'min_as']].astype(str)
                                         .apply(lambda x: '-'.join(x.str.zfill(2)), axis=1) + '-' +
                                         train['sec_as'].apply(lambda x: str(x).split('.')[0]).str.zfill(2)) + \
                         pd.to_timedelta(train['sec_as'].apply(lambda x: str(x).split('.')[1] if '.' in str(x) else '000'), unit='s')

train['delta_seconds'] = (train['aftershock_dt'] - train['main_dt_ms']).dt.total_seconds()


<ipython-input-26-80234700dccb>:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  train['main_dt'] = pd.to_datetime(train[['year', 'month', 'day', 'hour', 'min', 'sec']].astype(str)


DateParseError: Unknown datetime string format, unable to parse: 1980-02-15-09-09-350-350, at position 0

In [ ]:
pip install numpy pandas scikit-learn catboost pytorch-tabnet torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_absolute_error
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostRegressor
from pytorch_tabnet.tab_model import TabNetRegressor
import torch
import torch.nn as nn


train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

# Haversine distance
def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    return R * 2 * np.arcsin(np.sqrt(a))

# Feature engineering
def add_features(df, is_train=True):
    df['datetime'] = pd.to_datetime(dict(year=df['year'], month=df['month'], day=df['day'],
                                         hour=df['hour'], minute=df['min'], second=df['sec']), errors='coerce')
    df['day_of_year'] = df['datetime'].dt.dayofyear.fillna(0)
    df['main_eq_time_decimal'] = df['hour'] + df['min'] / 60 + df['sec'] / 3600
    df['cos_hour'] = np.cos(2 * np.pi * df['main_eq_time_decimal'] / 24)
    df['sin_hour'] = np.sin(2 * np.pi * df['main_eq_time_decimal'] / 24)
    df = df.replace([np.inf, -np.inf], np.nan).fillna(0)

    if is_train:
        df['distance_km'] = haversine(df['lat'], df['lon'], df['lat_as'], df['lon_as'])
    else:
        df['distance_km'] = 0

    return df

train = add_features(train, is_train=True)
test = add_features(test, is_train=False)

features = [
    "year", "month", "day", "hour", "min", "sec", "lat", "lon", "depth", "class",
    "day_of_year", "main_eq_time_decimal", "cos_hour", "sin_hour", "distance_km"
]
targets = [
    "year_as", "month_as", "day_as", "hour_as", "min_as", "sec_as",
    "lat_as", "lon_as", "depth_as", "class_as"
]

X = train[features].copy()
y = train[targets].copy()
X_test = test[features].copy()

scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)
X_test_scaled = scaler_X.transform(X_test)

# === PyTorch Model ===
class PyTorchModel(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 128), nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(128, 64), nn.ReLU(),
            nn.Linear(64, output_size)
        )

    def forward(self, x):
        return self.model(x)

# === Stacking ===
oof_preds_tabnet = np.zeros_like(y_scaled)
oof_preds_cat = np.zeros_like(y_scaled)
oof_preds_pytorch = np.zeros_like(y_scaled)

kf = KFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(kf.split(X_scaled)):
    print(f"Fold {fold+1}")
    X_train, X_val = X_scaled[train_idx], X_scaled[val_idx]
    y_train, y_val = y_scaled[train_idx], y_scaled[val_idx]

    # TabNet
    tabnet = TabNetRegressor(n_d=16, n_a=123, n_steps=3, gamma=1.5, lambda_sparse=1e-4,
                              optimizer_fn=torch.optim.Adam, optimizer_params=dict(lr=0.009),
                              verbose=0)
    tabnet.fit(X_train, y_train, eval_set=[(X_val, y_val)], patience=30)
    oof_preds_tabnet[val_idx] = tabnet.predict(X_val)

    # CatBoost
    cat = MultiOutputRegressor(CatBoostRegressor(verbose=0, task_type='GPU', iterations=500))
    cat.fit(X_train, y_train)
    oof_preds_cat[val_idx] = cat.predict(X_val)

    # PyTorch
    model = PyTorchModel(input_size=X_train.shape[1], output_size=y_train.shape[1])
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    loss_fn = nn.MSELoss()

    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
    X_val_tensor = torch.tensor(X_val, dtype=torch.float32)

    for epoch in range(50):
        model.train()
        optimizer.zero_grad()
        loss = loss_fn(model(X_train_tensor), y_train_tensor)
        loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        preds = model(X_val_tensor).numpy()
        oof_preds_pytorch[val_idx] = preds

# === Meta Model ===
X_meta = np.hstack([oof_preds_tabnet, oof_preds_cat, oof_preds_pytorch])
meta_model = MultiOutputRegressor(RidgeCV())
meta_model.fit(X_meta, y_scaled)

# === Final Prediction ===
tabnet.fit(X_scaled, y_scaled)
preds_tabnet = tabnet.predict(X_test_scaled)

cat.fit(X_scaled, y_scaled)
preds_cat = cat.predict(X_test_scaled)

X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y_scaled, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)

model = PyTorchModel(input_size=X_scaled.shape[1], output_size=y_scaled.shape[1])
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()
for epoch in range(50):
    model.train()
    optimizer.zero_grad()
    loss = loss_fn(model(X_tensor), y_tensor)
    loss.backward()
    optimizer.step()

model.eval()
with torch.no_grad():
    preds_pytorch = model(X_test_tensor).numpy()

X_test_meta = np.hstack([preds_tabnet, preds_cat, preds_pytorch])
final_preds_scaled = meta_model.predict(X_test_meta)
final_preds = scaler_y.inverse_transform(final_preds_scaled)

for i, col in enumerate(targets):
    test[col] = final_preds[:, i]

test["year_as"] = test["year_as"].round().astype(int)
test["month_as"] = test["month_as"].round().astype(int)
test["day_as"] = test["day_as"].round().astype(int)
test["hour_as"] = test["hour_as"].round().astype(int)
test["min_as"] = test["min_as"].round().astype(int)
test["sec_as"] = test["sec_as"].round(3)
test["lat_as"] = test["lat_as"].round(5)
test["lon_as"] = test["lon_as"].round(5)
test["depth_as"] = test["depth_as"].round(2)
test["class_as"] = test["class_as"].round(2)

submission = test[["id_eq"] + targets]
submission.to_csv("submission_stacked_with_distance.csv", index=False)
print("\n submission_stacked_with_distance.csv сохранён!")


Fold 1

Early stopping occurred at epoch 30 with best_epoch = 0 and best_val_0_mse = 163.56117


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Fold 2

Early stopping occurred at epoch 30 with best_epoch = 0 and best_val_0_mse = 279.08542


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Fold 3

Early stopping occurred at epoch 30 with best_epoch = 0 and best_val_0_mse = 176.18492


...

Fold 4

Early stopping occurred at epoch 30 with best_epoch = 0 and best_val_0_mse = 314.81712


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Fold 5

Early stopping occurred at epoch 30 with best_epoch = 0 and best_val_0_mse = 202.03294


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)



✅ submission_stacked_with_distance.csv сохранён!


In [ ]:
!pip install numpy==1.23.5 --quiet
import os
os.kill(os.getpid(), 9)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.linear_model import RidgeCV
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostRegressor
from pytorch_tabnet.tab_model import TabNetRegressor
import torch
import torch.nn as nn

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
if 'Unnamed: 0' in train.columns:
    train.drop(columns=['Unnamed: 0'], inplace=True)

def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    return R * 2 * np.arcsin(np.sqrt(a))

def add_features(df, is_train=True):
    df['datetime'] = pd.to_datetime(dict(year=df['year'], month=df['month'], day=df['day'],
                                         hour=df['hour'], minute=df['min'], second=df['sec']), errors='coerce')
    df['day_of_year'] = df['datetime'].dt.dayofyear.fillna(0)
    df['julian_day'] = df['datetime'].dt.strftime('%j').astype(float)
    df['main_eq_time_decimal'] = df['hour'] + df['min'] / 60 + df['sec'] / 3600
    df['cos_hour'] = np.cos(2 * np.pi * df['main_eq_time_decimal'] / 24)
    df['sin_hour'] = np.sin(2 * np.pi * df['main_eq_time_decimal'] / 24)
    df['is_night'] = ((df['hour'] < 6) | (df['hour'] >= 22)).astype(int)
    if is_train:
        df['distance_km'] = haversine(df['lat'], df['lon'], df['lat_as'], df['lon_as'])
    else:
        df['distance_km'] = train['distance_km'].mean()
    df = df.replace([np.inf, -np.inf], np.nan).fillna(0)
    return df

train = add_features(train, is_train=True)
test = add_features(test, is_train=False)

features = [
    "year", "month", "day", "hour", "min", "sec", "lat", "lon", "depth", "class",
    "day_of_year", "julian_day", "main_eq_time_decimal", "cos_hour", "sin_hour",
    "is_night", "distance_km"
]
targets = [
    "year_as", "month_as", "day_as", "hour_as", "min_as", "sec_as",
    "lat_as", "lon_as", "depth_as", "class_as"
]

X = train[features].copy()
y = train[targets].copy()
X_test = test[features].copy()

scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)
X_test_scaled = scaler_X.transform(X_test)

# === PyTorch модель ===
class PyTorchModel(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 128), nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(128, 64), nn.ReLU(),
            nn.Linear(64, output_size)
        )
    def forward(self, x):
        return self.model(x)


oof_preds_tabnet = np.zeros_like(y_scaled)
oof_preds_cat = np.zeros_like(y_scaled)
oof_preds_pytorch = np.zeros_like(y_scaled)

kf = KFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(kf.split(X_scaled)):
    print(f"\nFold {fold + 1}")
    X_train, X_val = X_scaled[train_idx], X_scaled[val_idx]
    y_train, y_val = y_scaled[train_idx], y_scaled[val_idx]

    # === TabNet ===
    tabnet = TabNetRegressor(
        n_d=16, n_a=123, n_steps=3, gamma=1.5, lambda_sparse=1e-4,
        optimizer_fn=torch.optim.Adam, optimizer_params=dict(lr=0.009),
        verbose=0, seed=42
    )
    tabnet.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        patience=30,
    )
    oof_preds_tabnet[val_idx] = tabnet.predict(X_val)

    # === CatBoost ===
    cat = MultiOutputRegressor(CatBoostRegressor(verbose=0, task_type='GPU', iterations=500))
    cat.fit(X_train, y_train)
    oof_preds_cat[val_idx] = cat.predict(X_val)

    # === PyTorch ===
    model = PyTorchModel(input_size=X_train.shape[1], output_size=y_train.shape[1])
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    loss_fn = nn.MSELoss()

    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
    X_val_tensor = torch.tensor(X_val, dtype=torch.float32)

    for epoch in range(50):
        model.train()
        optimizer.zero_grad()
        loss = loss_fn(model(X_train_tensor), y_train_tensor)
        loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        preds = model(X_val_tensor).numpy()
        oof_preds_pytorch[val_idx] = preds


X_meta = np.hstack([oof_preds_tabnet, oof_preds_cat, oof_preds_pytorch])
meta_model = MultiOutputRegressor(RidgeCV())
meta_model.fit(X_meta, y_scaled)

final_tabnet = TabNetRegressor(
    n_d=16, n_a=123, n_steps=3, gamma=1.5, lambda_sparse=1e-4,
    optimizer_fn=torch.optim.Adam, optimizer_params=dict(lr=0.009),
    verbose=0, seed=42
)
final_tabnet.fit(X_scaled, y_scaled, eval_metric='mae')
preds_tabnet = final_tabnet.predict(X_test_scaled)

cat.fit(X_scaled, y_scaled)
preds_cat = cat.predict(X_test_scaled)

X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y_scaled, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)

model = PyTorchModel(input_size=X_scaled.shape[1], output_size=y_scaled.shape[1])
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()
for epoch in range(50):
    model.train()
    optimizer.zero_grad()
    loss = loss_fn(model(X_tensor), y_tensor)
    loss.backward()
    optimizer.step()

model.eval()
with torch.no_grad():
    preds_pytorch = model(X_test_tensor).numpy()

X_test_meta = np.hstack([preds_tabnet, preds_cat, preds_pytorch])
final_preds_scaled = meta_model.predict(X_test_meta)
final_preds = scaler_y.inverse_transform(final_preds_scaled)

for i, col in enumerate(targets):
    test[col] = final_preds[:, i]

test["year_as"] = test["year_as"].round().astype(int)
test["month_as"] = test["month_as"].round().astype(int)
test["day_as"] = test["day_as"].round().astype(int)
test["hour_as"] = test["hour_as"].round().astype(int)
test["min_as"] = test["min_as"].round().astype(int)
test["sec_as"] = test["sec_as"].round(3)
test["lat_as"] = test["lat_as"].round(5)
test["lon_as"] = test["lon_as"].round(5)
test["depth_as"] = test["depth_as"].round(2)
test["class_as"] = test["class_as"].round(2)

submission = test[["id_eq"] + targets]
submission.to_csv("submission_stacked_improved.csv", index=False)
print("\n submission_stacked_improved.csv сохранён!")


Fold 1

Early stopping occurred at epoch 30 with best_epoch = 0 and best_val_0_mse = 182.51153


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



Fold 2

Early stopping occurred at epoch 30 with best_epoch = 0 and best_val_0_mse = 188.43981


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



Fold 3

Early stopping occurred at epoch 30 with best_epoch = 0 and best_val_0_mse = 233.34693


...


Fold 4

Early stopping occurred at epoch 30 with best_epoch = 0 and best_val_0_mse = 203.44734


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



Fold 5

Early stopping occurred at epoch 30 with best_epoch = 0 and best_val_0_mse = 182.23031


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)



✅ submission_stacked_improved.csv сохранён!


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.linear_model import RidgeCV
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostRegressor
from pytorch_tabnet.tab_model import TabNetRegressor
import torch
import torch.nn as nn

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
if 'Unnamed: 0' in train.columns:
    train.drop(columns=['Unnamed: 0'], inplace=True)


def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    return R * 2 * np.arcsin(np.sqrt(a))

def add_features(df, is_train=True):
    df['datetime'] = pd.to_datetime(dict(year=df['year'], month=df['month'], day=df['day'],
                                         hour=df['hour'], minute=df['min'], second=df['sec']), errors='coerce')
    df['day_of_year'] = df['datetime'].dt.dayofyear.fillna(0)
    df['julian_day'] = df['datetime'].dt.strftime('%j').astype(float)
    df['main_eq_time_decimal'] = df['hour'] + df['min'] / 60 + df['sec'] / 3600
    df['cos_hour'] = np.cos(2 * np.pi * df['main_eq_time_decimal'] / 24)
    df['sin_hour'] = np.sin(2 * np.pi * df['main_eq_time_decimal'] / 24)
    df['is_night'] = ((df['hour'] < 6) | (df['hour'] >= 22)).astype(int)
    if is_train:
        df['distance_km'] = haversine(df['lat'], df['lon'], df['lat_as'], df['lon_as'])
    else:
        df['distance_km'] = train['distance_km'].mean()
    df = df.replace([np.inf, -np.inf], np.nan).fillna(0)
    return df

train = add_features(train, is_train=True)
test = add_features(test, is_train=False)

features = [
    "year", "month", "day", "hour", "min", "sec", "lat", "lon", "depth", "class",
    "day_of_year", "julian_day", "main_eq_time_decimal", "cos_hour", "sin_hour",
    "is_night", "distance_km"
]
targets = [
    "year_as", "month_as", "day_as", "hour_as", "min_as", "sec_as",
    "lat_as", "lon_as", "depth_as", "class_as"
]

X = train[features].copy()
y = train[targets].copy()
X_test = test[features].copy()

log_targets = ['sec_as', 'depth_as', 'class_as']
log_target_indices = [targets.index(col) for col in log_targets]

for col in log_targets:
    y[col] = np.where(y[col] <= 0, 1e-6, y[col])
    y[col] = np.log1p(y[col])

scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)
X_test_scaled = scaler_X.transform(X_test)

class PyTorchModel(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 128), nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(128, 64), nn.ReLU(),
            nn.Linear(64, output_size)
        )
    def forward(self, x):
        return self.model(x)

oof_preds_tabnet = np.zeros_like(y_scaled)
oof_preds_cat = np.zeros_like(y_scaled)
oof_preds_pytorch = np.zeros_like(y_scaled)

kf = KFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(kf.split(X_scaled)):
    print(f"\nFold {fold + 1}")
    X_train, X_val = X_scaled[train_idx], X_scaled[val_idx]
    y_train, y_val = y_scaled[train_idx], y_scaled[val_idx]

    # === TabNet (упрощённый) ===
    tabnet = TabNetRegressor(
        n_d=8, n_a=16, n_steps=2, gamma=1.2, lambda_sparse=1e-4,
        optimizer_fn=torch.optim.Adam, optimizer_params=dict(lr=0.01),
        verbose=0, seed=42
    )
    tabnet.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        patience=30
    )
    oof_preds_tabnet[val_idx] = tabnet.predict(X_val)

    # === CatBoost ===
    cat = MultiOutputRegressor(CatBoostRegressor(verbose=0, task_type='GPU', iterations=500))
    cat.fit(X_train, y_train)
    oof_preds_cat[val_idx] = cat.predict(X_val)

    # === PyTorch ===
    model = PyTorchModel(input_size=X_train.shape[1], output_size=y_train.shape[1])
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    loss_fn = nn.MSELoss()

    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
    X_val_tensor = torch.tensor(X_val, dtype=torch.float32)

    for epoch in range(50):
        model.train()
        optimizer.zero_grad()
        loss = loss_fn(model(X_train_tensor), y_train_tensor)
        loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        preds = model(X_val_tensor).numpy()
        oof_preds_pytorch[val_idx] = preds

X_meta = np.hstack([oof_preds_tabnet, oof_preds_cat, oof_preds_pytorch])
meta_model = MultiOutputRegressor(RidgeCV())
meta_model.fit(X_meta, y_scaled)

final_tabnet = TabNetRegressor(
    n_d=8, n_a=16, n_steps=2, gamma=1.2, lambda_sparse=1e-4,
    optimizer_fn=torch.optim.Adam, optimizer_params=dict(lr=0.01),
    verbose=0, seed=42
)
final_tabnet.fit(X_scaled, y_scaled)
preds_tabnet = final_tabnet.predict(X_test_scaled)

cat.fit(X_scaled, y_scaled)
preds_cat = cat.predict(X_test_scaled)

X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y_scaled, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)

model = PyTorchModel(input_size=X_scaled.shape[1], output_size=y_scaled.shape[1])
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()
for epoch in range(50):
    model.train()
    optimizer.zero_grad()
    loss = loss_fn(model(X_tensor), y_tensor)
    loss.backward()
    optimizer.step()

model.eval()
with torch.no_grad():
    preds_pytorch = model(X_test_tensor).numpy()

X_test_meta = np.hstack([preds_tabnet, preds_cat, preds_pytorch])
final_preds_scaled = meta_model.predict(X_test_meta)
final_preds = scaler_y.inverse_transform(final_preds_scaled)


final_preds[:, log_target_indices] = np.expm1(final_preds[:, log_target_indices])

for i, col in enumerate(targets):
    test[col] = final_preds[:, i]

test["year_as"] = test["year_as"].round().astype(int)
test["month_as"] = test["month_as"].round().astype(int)
test["day_as"] = test["day_as"].round().astype(int)
test["hour_as"] = test["hour_as"].round().astype(int)
test["min_as"] = test["min_as"].round().astype(int)
test["sec_as"] = test["sec_as"].round(3)
test["lat_as"] = test["lat_as"].round(5)
test["lon_as"] = test["lon_as"].round(5)
test["depth_as"] = test["depth_as"].round(2)
test["class_as"] = test["class_as"].round(2)

submission = test[["id_eq"] + targets]
submission.to_csv("submission_stacked_log_scaled.csv", index=False)
print("\n submission_stacked_log_scaled.csv сохранён!")


Fold 1

Early stopping occurred at epoch 30 with best_epoch = 0 and best_val_0_mse = 5.5832


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



Fold 2

Early stopping occurred at epoch 30 with best_epoch = 0 and best_val_0_mse = 5.40245


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



Fold 3

Early stopping occurred at epoch 30 with best_epoch = 0 and best_val_0_mse = 5.63884


...


Fold 4

Early stopping occurred at epoch 30 with best_epoch = 0 and best_val_0_mse = 5.25247


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



Fold 5

Early stopping occurred at epoch 30 with best_epoch = 0 and best_val_0_mse = 5.02328


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)



✅ submission_stacked_log_scaled.csv сохранён!


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.linear_model import RidgeCV
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostRegressor
from pytorch_tabnet.tab_model import TabNetRegressor
import torch
import torch.nn as nn


train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
if 'Unnamed: 0' in train.columns:
    train.drop(columns=['Unnamed: 0'], inplace=True)

def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    return R * 2 * np.arcsin(np.sqrt(a))

def add_features(df, is_train=True):
    df['datetime'] = pd.to_datetime(dict(year=df['year'], month=df['month'], day=df['day'],
                                         hour=df['hour'], minute=df['min'], second=df['sec']), errors='coerce')
    df['day_of_year'] = df['datetime'].dt.dayofyear.fillna(0)
    df['julian_day'] = df['datetime'].dt.strftime('%j').astype(float)
    df['main_eq_time_decimal'] = df['hour'] + df['min'] / 60 + df['sec'] / 3600
    df['cos_hour'] = np.cos(2 * np.pi * df['main_eq_time_decimal'] / 24)
    df['sin_hour'] = np.sin(2 * np.pi * df['main_eq_time_decimal'] / 24)
    df['is_night'] = ((df['hour'] < 6) | (df['hour'] >= 22)).astype(int)
    if is_train:
        df['distance_km'] = haversine(df['lat'], df['lon'], df['lat_as'], df['lon_as'])
    else:
        df['distance_km'] = train['distance_km'].mean()
    df = df.replace([np.inf, -np.inf], np.nan).fillna(0)
    return df

train = add_features(train, is_train=True)
test = add_features(test, is_train=False)

features = [
    "year", "month", "day", "hour", "min", "sec", "lat", "lon", "depth", "class",
    "day_of_year", "julian_day", "main_eq_time_decimal", "cos_hour", "sin_hour",
    "is_night", "distance_km"
]
targets = [
    "year_as", "month_as", "day_as", "hour_as", "min_as", "sec_as",
    "lat_as", "lon_as", "depth_as", "class_as"
]

X = train[features].copy()
y = train[targets].copy()
X_test = test[features].copy()

log_targets = ['sec_as', 'depth_as', 'class_as']
log_target_indices = [targets.index(col) for col in log_targets]

for col in log_targets:
    y[col] = np.where(y[col] <= 0, 1e-6, y[col])
    y[col] = np.log1p(y[col])

scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)
X_test_scaled = scaler_X.transform(X_test)

class PyTorchModel(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 128), nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(128, 64), nn.ReLU(),
            nn.Linear(64, output_size)
        )
    def forward(self, x):
        return self.model(x)


oof_preds_tabnet = np.zeros_like(y_scaled)
oof_preds_cat = np.zeros_like(y_scaled)
oof_preds_pytorch = np.zeros_like(y_scaled)

kf = KFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(kf.split(X_scaled)):
    print(f"\nFold {fold + 1}")
    X_train, X_val = X_scaled[train_idx], X_scaled[val_idx]
    y_train, y_val = y_scaled[train_idx], y_scaled[val_idx]

    tabnet = TabNetRegressor(
        n_d=8, n_a=16, n_steps=2, gamma=1.2, lambda_sparse=1e-4,
        optimizer_fn=torch.optim.Adam, optimizer_params=dict(lr=0.01),
        verbose=0, seed=42
    )
    tabnet.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        patience=30
    )
    oof_preds_tabnet[val_idx] = tabnet.predict(X_val)

    # === CatBoost ===
    cat = MultiOutputRegressor(CatBoostRegressor(verbose=0, task_type='GPU', iterations=500))
    cat.fit(X_train, y_train)
    oof_preds_cat[val_idx] = cat.predict(X_val)

    # === PyTorch ===
    model = PyTorchModel(input_size=X_train.shape[1], output_size=y_train.shape[1])
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    loss_fn = nn.MSELoss()

    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
    X_val_tensor = torch.tensor(X_val, dtype=torch.float32)

    for epoch in range(50):
        model.train()
        optimizer.zero_grad()
        loss = loss_fn(model(X_train_tensor), y_train_tensor)
        loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        preds = model(X_val_tensor).numpy()
        oof_preds_pytorch[val_idx] = preds


X_meta = np.hstack([oof_preds_tabnet, oof_preds_cat, oof_preds_pytorch])
meta_model = MultiOutputRegressor(RidgeCV())
meta_model.fit(X_meta, y_scaled)

final_tabnet = TabNetRegressor(
    n_d=8, n_a=16, n_steps=2, gamma=1.2, lambda_sparse=1e-4,
    optimizer_fn=torch.optim.Adam, optimizer_params=dict(lr=0.01),
    verbose=0, seed=42
)
final_tabnet.fit(X_scaled, y_scaled)
preds_tabnet = final_tabnet.predict(X_test_scaled)

cat.fit(X_scaled, y_scaled)
preds_cat = cat.predict(X_test_scaled)

X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y_scaled, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)

model = PyTorchModel(input_size=X_scaled.shape[1], output_size=y_scaled.shape[1])
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()
for epoch in range(50):
    model.train()
    optimizer.zero_grad()
    loss = loss_fn(model(X_tensor), y_tensor)
    loss.backward()
    optimizer.step()

model.eval()
with torch.no_grad():
    preds_pytorch = model(X_test_tensor).numpy()

X_test_meta = np.hstack([preds_tabnet, preds_cat, preds_pytorch])
final_preds_scaled = meta_model.predict(X_test_meta)
final_preds = scaler_y.inverse_transform(final_preds_scaled)

final_preds[:, log_target_indices] = np.expm1(final_preds[:, log_target_indices])

for i, col in enumerate(targets):
    test[col] = final_preds[:, i]

test["year_as"] = test["year_as"].round().astype(int)
test["month_as"] = test["month_as"].round().astype(int)
test["day_as"] = test["day_as"].round().astype(int)
test["hour_as"] = test["hour_as"].round().astype(int)
test["min_as"] = test["min_as"].round().astype(int)
test["sec_as"] = test["sec_as"].round(3)
test["lat_as"] = test["lat_as"].round(5)
test["lon_as"] = test["lon_as"].round(5)
test["depth_as"] = test["depth_as"].round(2)
test["class_as"] = test["class_as"].round(2)

submission = test[["id_eq"] + targets]
submission.to_csv("submission_stacked_log_scaled.csv", index=False)
print("\n submission_stacked_log_scaled.csv сохранён!")


Fold 1

Early stopping occurred at epoch 30 with best_epoch = 0 and best_val_0_mse = 5.5832


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



Fold 2

Early stopping occurred at epoch 30 with best_epoch = 0 and best_val_0_mse = 5.40245


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



Fold 3

Early stopping occurred at epoch 30 with best_epoch = 0 and best_val_0_mse = 5.63884


...


Fold 4

Early stopping occurred at epoch 30 with best_epoch = 0 and best_val_0_mse = 5.25247


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



Fold 5

Early stopping occurred at epoch 30 with best_epoch = 0 and best_val_0_mse = 5.02328


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)



✅ submission_stacked_log_scaled.csv сохранён!


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.linear_model import RidgeCV
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostRegressor
from pytorch_tabnet.tab_model import TabNetRegressor
import torch
import torch.nn as nn


train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
if 'Unnamed: 0' in train.columns:
    train.drop(columns=['Unnamed: 0'], inplace=True)

def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    return R * 2 * np.arcsin(np.sqrt(a))

def add_features(df, is_train=True):
    df['datetime'] = pd.to_datetime(dict(year=df['year'], month=df['month'], day=df['day'],
                                         hour=df['hour'], minute=df['min'], second=df['sec']), errors='coerce')
    df['day_of_year'] = df['datetime'].dt.dayofyear.fillna(0)
    df['julian_day'] = df['datetime'].dt.strftime('%j').astype(float)
    df['main_eq_time_decimal'] = df['hour'] + df['min'] / 60 + df['sec'] / 3600
    df['cos_hour'] = np.cos(2 * np.pi * df['main_eq_time_decimal'] / 24)
    df['sin_hour'] = np.sin(2 * np.pi * df['main_eq_time_decimal'] / 24)
    df['is_night'] = ((df['hour'] < 6) | (df['hour'] >= 22)).astype(int)
    df['hour_fraction'] = df['main_eq_time_decimal'] / 24
    df['season'] = ((df['month'] % 12 + 3) // 3).astype(int)
    df['is_weekend'] = df['datetime'].dt.weekday >= 5
    if is_train:
        df['distance_km'] = haversine(df['lat'], df['lon'], df['lat_as'], df['lon_as'])
        df['delta_lat'] = df['lat_as'] - df['lat']
        df['delta_lon'] = df['lon_as'] - df['lon']
        df['delta_depth'] = df['depth_as'] - df['depth']
    else:
        df['distance_km'] = train['distance_km'].mean()
        df['delta_lat'] = 0
        df['delta_lon'] = 0
        df['delta_depth'] = 0
    df = df.replace([np.inf, -np.inf], np.nan).fillna(0)
    return df

train = add_features(train, is_train=True)
test = add_features(test, is_train=False)

features = [
    "year", "month", "day", "hour", "min", "sec", "lat", "lon", "depth", "class",
    "day_of_year", "julian_day", "main_eq_time_decimal", "cos_hour", "sin_hour",
    "is_night", "distance_km", "hour_fraction", "season", "is_weekend",
    "delta_lat", "delta_lon", "delta_depth"
]
targets = [
    "year_as", "month_as", "day_as", "hour_as", "min_as", "sec_as",
    "lat_as", "lon_as", "depth_as", "class_as"
]

X = train[features].copy()
y = train[targets].copy()
X_test = test[features].copy()

scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)
X_test_scaled = scaler_X.transform(X_test)

# === PyTorch  ===
class PyTorchModel(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 128), nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(128, 64), nn.ReLU(),
            nn.Linear(64, output_size)
        )
    def forward(self, x):
        return self.model(x)

oof_preds_tabnet = np.zeros_like(y_scaled)
oof_preds_cat = np.zeros_like(y_scaled)
oof_preds_pytorch = np.zeros_like(y_scaled)

kf = KFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(kf.split(X_scaled)):
    print(f"\nFold {fold + 1}")
    X_train, X_val = X_scaled[train_idx], X_scaled[val_idx]
    y_train, y_val = y_scaled[train_idx], y_scaled[val_idx]

    tabnet = TabNetRegressor(
        n_d=16, n_a=123, n_steps=3, gamma=1.5, lambda_sparse=1e-4,
        optimizer_fn=torch.optim.Adam, optimizer_params=dict(lr=0.009),
        verbose=0, seed=42
    )
    tabnet.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        patience=30
    )
    oof_preds_tabnet[val_idx] = tabnet.predict(X_val)

    cat = MultiOutputRegressor(CatBoostRegressor(verbose=0, task_type='GPU', iterations=500))
    cat.fit(X_train, y_train)
    oof_preds_cat[val_idx] = cat.predict(X_val)

    model = PyTorchModel(input_size=X_train.shape[1], output_size=y_train.shape[1])
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    loss_fn = nn.MSELoss()

    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
    X_val_tensor = torch.tensor(X_val, dtype=torch.float32)

    for epoch in range(50):
        model.train()
        optimizer.zero_grad()
        loss = loss_fn(model(X_train_tensor), y_train_tensor)
        loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        preds = model(X_val_tensor).numpy()
        oof_preds_pytorch[val_idx] = preds

X_meta = np.hstack([oof_preds_tabnet, oof_preds_cat, oof_preds_pytorch])
meta_model = MultiOutputRegressor(RidgeCV())
meta_model.fit(X_meta, y_scaled)

final_tabnet = TabNetRegressor(
    n_d=16, n_a=123, n_steps=3, gamma=1.5, lambda_sparse=1e-4,
    optimizer_fn=torch.optim.Adam, optimizer_params=dict(lr=0.009),
    verbose=0, seed=42
)
final_tabnet.fit(X_scaled, y_scaled)
preds_tabnet = final_tabnet.predict(X_test_scaled)

cat.fit(X_scaled, y_scaled)
preds_cat = cat.predict(X_test_scaled)

X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y_scaled, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)

model = PyTorchModel(input_size=X_scaled.shape[1], output_size=y_scaled.shape[1])
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()
for epoch in range(50):
    model.train()
    optimizer.zero_grad()
    loss = loss_fn(model(X_tensor), y_tensor)
    loss.backward()
    optimizer.step()

model.eval()
with torch.no_grad():
    preds_pytorch = model(X_test_tensor).numpy()

X_test_meta = np.hstack([preds_tabnet, preds_cat, preds_pytorch])
final_preds_scaled = meta_model.predict(X_test_meta)
final_preds = scaler_y.inverse_transform(final_preds_scaled)

for i, col in enumerate(targets):
    test[col] = final_preds[:, i]

test["year_as"] = test["year_as"].round().astype(int)
test["month_as"] = test["month_as"].round().astype(int)
test["day_as"] = test["day_as"].round().astype(int)
test["hour_as"] = test["hour_as"].round().astype(int)
test["min_as"] = test["min_as"].round().astype(int)
test["sec_as"] = test["sec_as"].round(3)
test["lat_as"] = test["lat_as"].round(5)
test["lon_as"] = test["lon_as"].round(5)
test["depth_as"] = test["depth_as"].round(2)
test["class_as"] = test["class_as"].round(2)

submission = test[["id_eq"] + targets]
submission.to_csv("submission_stacked_features.csv", index=False)
print("\n submission_stacked_features.csv сохранён!")



Fold 1

Early stopping occurred at epoch 30 with best_epoch = 0 and best_val_0_mse = 117.98399


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



Fold 2

Early stopping occurred at epoch 30 with best_epoch = 0 and best_val_0_mse = 155.62956


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



Fold 3

Early stopping occurred at epoch 30 with best_epoch = 0 and best_val_0_mse = 151.25207


...


Fold 4

Early stopping occurred at epoch 30 with best_epoch = 0 and best_val_0_mse = 132.30688


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



Fold 5

Early stopping occurred at epoch 30 with best_epoch = 0 and best_val_0_mse = 144.20659


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)



✅ submission_stacked_features.csv сохранён!


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.linear_model import RidgeCV
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostRegressor
from pytorch_tabnet.tab_model import TabNetRegressor
import torch
import torch.nn as nn


train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
if 'Unnamed: 0' in train.columns:
    train.drop(columns=['Unnamed: 0'], inplace=True)

def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    return R * 2 * np.arcsin(np.sqrt(a))

def add_features(df, is_train=True):
    df['datetime'] = pd.to_datetime(dict(year=df['year'], month=df['month'], day=df['day'],
                                         hour=df['hour'], minute=df['min'], second=df['sec']), errors='coerce')
    df['day_of_year'] = df['datetime'].dt.dayofyear.fillna(0)
    df['julian_day'] = df['datetime'].dt.strftime('%j').astype(float)
    df['main_eq_time_decimal'] = df['hour'] + df['min'] / 60 + df['sec'] / 3600
    df['cos_hour'] = np.cos(2 * np.pi * df['main_eq_time_decimal'] / 24)
    df['sin_hour'] = np.sin(2 * np.pi * df['main_eq_time_decimal'] / 24)
    df['is_night'] = ((df['hour'] < 6) | (df['hour'] >= 22)).astype(int)
    df['hour_fraction'] = df['main_eq_time_decimal'] / 24
    df['season'] = ((df['month'] % 12 + 3) // 3).astype(int)
    df['is_weekend'] = df['datetime'].dt.weekday >= 5
    if is_train:
        df['distance_km'] = haversine(df['lat'], df['lon'], df['lat_as'], df['lon_as'])
        df['delta_lat'] = df['lat_as'] - df['lat']
        df['delta_lon'] = df['lon_as'] - df['lon']
        df['delta_depth'] = df['depth_as'] - df['depth']
    else:
        df['distance_km'] = train['distance_km'].mean()
        df['delta_lat'] = 0
        df['delta_lon'] = 0
        df['delta_depth'] = 0
    df = df.replace([np.inf, -np.inf], np.nan).fillna(0)
    return df

train = add_features(train, is_train=True)
test = add_features(test, is_train=False)

features = [
    "year", "month", "day", "hour", "min", "sec", "lat", "lon", "depth", "class",
    "day_of_year", "julian_day", "main_eq_time_decimal", "cos_hour", "sin_hour",
    "is_night", "distance_km", "hour_fraction", "season", "is_weekend",
    "delta_lat", "delta_lon", "delta_depth"
]

time_targets = ["year_as", "month_as", "day_as", "hour_as", "min_as", "sec_as"]
space_targets = ["lat_as", "lon_as", "depth_as", "class_as"]

y_time = train[time_targets].copy()
y_space = train[space_targets].copy()
X = train[features].copy()
X_test = test[features].copy()

scaler_X = StandardScaler()
scaler_y_time = StandardScaler()
scaler_y_space = StandardScaler()
X_scaled = scaler_X.fit_transform(X)
y_time_scaled = scaler_y_time.fit_transform(y_time)
y_space_scaled = scaler_y_space.fit_transform(y_space)
X_test_scaled = scaler_X.transform(X_test)

class PyTorchModel(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 128), nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(128, 64), nn.ReLU(),
            nn.Linear(64, output_size)
        )
    def forward(self, x):
        return self.model(x)

kf = KFold(n_splits=5, shuffle=True, random_state=42)
oof_preds_time = np.zeros_like(y_time_scaled)
oof_preds_space = np.zeros_like(y_space_scaled)

for fold, (train_idx, val_idx) in enumerate(kf.split(X_scaled)):
    print(f"\nFold {fold + 1}")
    X_train, X_val = X_scaled[train_idx], X_scaled[val_idx]
    y_train_time, y_val_time = y_time_scaled[train_idx], y_time_scaled[val_idx]
    y_train_space, y_val_space = y_space_scaled[train_idx], y_space_scaled[val_idx]

    # Time — CatBoost
    cat_time = MultiOutputRegressor(CatBoostRegressor(verbose=0, task_type='GPU', iterations=500))
    cat_time.fit(X_train, y_train_time)
    oof_preds_time[val_idx] = cat_time.predict(X_val)

    # Space — PyTorch
    model_space = PyTorchModel(input_size=X_train.shape[1], output_size=y_train_space.shape[1])
    optimizer = torch.optim.Adam(model_space.parameters(), lr=0.001)
    loss_fn = nn.MSELoss()

    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train_space, dtype=torch.float32)
    X_val_tensor = torch.tensor(X_val, dtype=torch.float32)

    for epoch in range(50):
        model_space.train()
        optimizer.zero_grad()
        loss = loss_fn(model_space(X_train_tensor), y_train_tensor)
        loss.backward()
        optimizer.step()

    model_space.eval()
    with torch.no_grad():
        preds = model_space(X_val_tensor).numpy()
        oof_preds_space[val_idx] = preds

X_meta = np.hstack([oof_preds_time, oof_preds_space])
y_all_scaled = np.hstack([y_time_scaled, y_space_scaled])
meta_model = MultiOutputRegressor(RidgeCV())
meta_model.fit(X_meta, y_all_scaled)

cat_time.fit(X_scaled, y_time_scaled)
model_space = PyTorchModel(input_size=X_scaled.shape[1], output_size=y_space_scaled.shape[1])
optimizer = torch.optim.Adam(model_space.parameters(), lr=0.001)
loss_fn = nn.MSELoss()
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y_space_scaled, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)

for epoch in range(50):
    model_space.train()
    optimizer.zero_grad()
    loss = loss_fn(model_space(X_tensor), y_tensor)
    loss.backward()
    optimizer.step()

model_space.eval()
with torch.no_grad():
    preds_space = model_space(X_test_tensor).numpy()

preds_time = cat_time.predict(X_test_scaled)
X_test_meta = np.hstack([preds_time, preds_space])
final_preds_scaled = meta_model.predict(X_test_meta)
final_preds = np.hstack([
    scaler_y_time.inverse_transform(final_preds_scaled[:, :len(time_targets)]),
    scaler_y_space.inverse_transform(final_preds_scaled[:, len(time_targets):])
])

for i, col in enumerate(time_targets + space_targets):
    test[col] = final_preds[:, i]

test["year_as"] = test["year_as"].round().astype(int)
test["month_as"] = test["month_as"].round().astype(int)
test["day_as"] = test["day_as"].round().astype(int)
test["hour_as"] = test["hour_as"].round().astype(int)
test["min_as"] = test["min_as"].round().astype(int)
test["sec_as"] = test["sec_as"].round(3)
test["lat_as"] = test["lat_as"].round(5)
test["lon_as"] = test["lon_as"].round(5)
test["depth_as"] = test["depth_as"].round(2)
test["class_as"] = test["class_as"].round(2)

submission = test[["id_eq"] + time_targets + space_targets]
submission.to_csv("submission_split_targets.csv", index=False)
print("\n submission_split_targets.csv сохранён!")


Fold 1

Fold 2

Fold 3

Fold 4

Fold 5

✅ submission_split_targets.csv сохранён!


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.linear_model import RidgeCV
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostRegressor
from pytorch_tabnet.tab_model import TabNetRegressor
import torch
import torch.nn as nn

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
if 'Unnamed: 0' in train.columns:
    train.drop(columns=['Unnamed: 0'], inplace=True)

def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    return R * 2 * np.arcsin(np.sqrt(a))

def add_features(df, is_train=True):
    df['datetime'] = pd.to_datetime(dict(year=df['year'], month=df['month'], day=df['day'],
                                         hour=df['hour'], minute=df['min'], second=df['sec']), errors='coerce')
    df['day_of_year'] = df['datetime'].dt.dayofyear.fillna(0)
    df['julian_day'] = df['datetime'].dt.strftime('%j').astype(float)
    df['main_eq_time_decimal'] = df['hour'] + df['min'] / 60 + df['sec'] / 3600
    df['cos_hour'] = np.cos(2 * np.pi * df['main_eq_time_decimal'] / 24)
    df['sin_hour'] = np.sin(2 * np.pi * df['main_eq_time_decimal'] / 24)
    df['is_night'] = ((df['hour'] < 6) | (df['hour'] >= 22)).astype(int)
    df['hour_fraction'] = df['main_eq_time_decimal'] / 24
    df['season'] = ((df['month'] % 12 + 3) // 3).astype(int)
    df['is_weekend'] = df['datetime'].dt.weekday >= 5
    if is_train:
        df['distance_km'] = haversine(df['lat'], df['lon'], df['lat_as'], df['lon_as'])
        df['delta_lat'] = df['lat_as'] - df['lat']
        df['delta_lon'] = df['lon_as'] - df['lon']
        df['delta_depth'] = df['depth_as'] - df['depth']
    else:
        df['distance_km'] = train['distance_km'].mean()
        df['delta_lat'] = 0
        df['delta_lon'] = 0
        df['delta_depth'] = 0
    df = df.replace([np.inf, -np.inf], np.nan).fillna(0)
    return df

train = add_features(train, is_train=True)
test = add_features(test, is_train=False)

features = [
    "year", "month", "day", "hour", "min", "sec", "lat", "lon", "depth", "class",
    "day_of_year", "julian_day", "main_eq_time_decimal", "cos_hour", "sin_hour",
    "is_night", "distance_km", "hour_fraction", "season", "is_weekend",
    "delta_lat", "delta_lon", "delta_depth"
]

time_targets = ["year_as", "month_as", "day_as", "hour_as", "min_as", "sec_as"]
space_targets = ["lat_as", "lon_as", "depth_as", "class_as"]

y_time = train[time_targets].copy()
y_space = train[space_targets].copy()
X = train[features].copy()
X_test = test[features].copy()

scaler_X = StandardScaler()
scaler_y_time = StandardScaler()
scaler_y_space = StandardScaler()
X_scaled = scaler_X.fit_transform(X)
y_time_scaled = scaler_y_time.fit_transform(y_time)
y_space_scaled = scaler_y_space.fit_transform(y_space)
X_test_scaled = scaler_X.transform(X_test)

class PyTorchModel(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 128), nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(128, 64), nn.ReLU(),
            nn.Linear(64, output_size)
        )
    def forward(self, x):
        return self.model(x)

kf = KFold(n_splits=5, shuffle=True, random_state=42)
oof_time = np.zeros_like(y_time_scaled)
oof_space_tabnet = np.zeros_like(y_space_scaled)
oof_space_pytorch = np.zeros_like(y_space_scaled)

for fold, (train_idx, val_idx) in enumerate(kf.split(X_scaled)):
    print(f"\nFold {fold + 1}")
    X_train, X_val = X_scaled[train_idx], X_scaled[val_idx]
    y_train_time, y_val_time = y_time_scaled[train_idx], y_time_scaled[val_idx]
    y_train_space, y_val_space = y_space_scaled[train_idx], y_space_scaled[val_idx]

    cat_time = MultiOutputRegressor(CatBoostRegressor(verbose=0, task_type='GPU', iterations=500))
    cat_time.fit(X_train, y_train_time)
    oof_time[val_idx] = cat_time.predict(X_val)

    tabnet = TabNetRegressor(
        n_d=8, n_a=16, n_steps=2, gamma=1.2, lambda_sparse=1e-4,
        optimizer_fn=torch.optim.Adam, optimizer_params=dict(lr=0.01),
        verbose=0, seed=42
    )
    tabnet.fit(X_train, y_train_space, eval_set=[(X_val, y_val_space)], patience=30)
    oof_space_tabnet[val_idx] = tabnet.predict(X_val)

    model = PyTorchModel(input_size=X_train.shape[1], output_size=y_train_space.shape[1])
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    loss_fn = nn.MSELoss()
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train_space, dtype=torch.float32)
    X_val_tensor = torch.tensor(X_val, dtype=torch.float32)

    for epoch in range(50):
        model.train()
        optimizer.zero_grad()
        loss = loss_fn(model(X_train_tensor), y_train_tensor)
        loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        oof_space_pytorch[val_idx] = model(X_val_tensor).numpy()

X_meta = np.hstack([oof_time, oof_space_tabnet, oof_space_pytorch])
y_all_scaled = np.hstack([y_time_scaled, y_space_scaled])
meta_model = MultiOutputRegressor(RidgeCV())
meta_model.fit(X_meta, y_all_scaled)

cat_time.fit(X_scaled, y_time_scaled)

tabnet = TabNetRegressor(
    n_d=8, n_a=16, n_steps=2, gamma=1.2, lambda_sparse=1e-4,
    optimizer_fn=torch.optim.Adam, optimizer_params=dict(lr=0.01),
    verbose=0, seed=42
)
tabnet.fit(X_scaled, y_space_scaled)
preds_tabnet = tabnet.predict(X_test_scaled)

model = PyTorchModel(input_size=X_scaled.shape[1], output_size=y_space_scaled.shape[1])
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y_space_scaled, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)

for epoch in range(50):
    model.train()
    optimizer.zero_grad()
    loss = loss_fn(model(X_tensor), y_tensor)
    loss.backward()
    optimizer.step()

model.eval()
with torch.no_grad():
    preds_pytorch = model(X_test_tensor).numpy()

preds_time = cat_time.predict(X_test_scaled)
X_test_meta = np.hstack([preds_time, preds_tabnet, preds_pytorch])
final_preds_scaled = meta_model.predict(X_test_meta)
final_preds = np.hstack([
    scaler_y_time.inverse_transform(final_preds_scaled[:, :len(time_targets)]),
    scaler_y_space.inverse_transform(final_preds_scaled[:, len(time_targets):])
])

for i, col in enumerate(time_targets + space_targets):
    test[col] = final_preds[:, i]

test["year_as"] = test["year_as"].round().astype(int)
test["month_as"] = test["month_as"].round().astype(int)
test["day_as"] = test["day_as"].round().astype(int)
test["hour_as"] = test["hour_as"].round().astype(int)
test["min_as"] = test["min_as"].round().astype(int)
test["sec_as"] = test["sec_as"].round(3)
test["lat_as"] = test["lat_as"].round(5)
test["lon_as"] = test["lon_as"].round(5)
test["depth_as"] = test["depth_as"].round(2)
test["class_as"] = test["class_as"].round(2)

submission = test[["id_eq"] + time_targets + space_targets]
submission.to_csv("submission_tabnet_pytorch_catboost.csv", index=False)
print("\n submission_tabnet_pytorch_catboost.csv сохранён!")



Fold 1

Early stopping occurred at epoch 30 with best_epoch = 0 and best_val_0_mse = 3.36429


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



Fold 2

Early stopping occurred at epoch 30 with best_epoch = 0 and best_val_0_mse = 4.38157


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



Fold 3

Early stopping occurred at epoch 30 with best_epoch = 0 and best_val_0_mse = 4.05067


...


Fold 4

Early stopping occurred at epoch 30 with best_epoch = 0 and best_val_0_mse = 3.65227


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



Fold 5

Early stopping occurred at epoch 30 with best_epoch = 0 and best_val_0_mse = 4.99777


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)
/usr/local/lib/python3.11/dist-packages/torch/utils/data/_utils/collate.py:285: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at /pytorch/torch/csrc/utils/tensor_numpy.cpp:203.)
  return collate([torch.as_tensor(b) for b in batch], collate_fn_map=collate_fn_map)



✅ submission_tabnet_pytorch_catboost.csv сохранён!


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.linear_model import RidgeCV
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostRegressor
from pytorch_tabnet.tab_model import TabNetRegressor
import torch
import torch.nn as nn

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
if 'Unnamed: 0' in train.columns:
    train.drop(columns=['Unnamed: 0'], inplace=True)

def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    return R * 2 * np.arcsin(np.sqrt(a))

def add_features(df, is_train=True):
    df['datetime'] = pd.to_datetime(dict(year=df['year'], month=df['month'], day=df['day'],
                                         hour=df['hour'], minute=df['min'], second=df['sec']), errors='coerce')
    df['day_of_year'] = df['datetime'].dt.dayofyear.fillna(0)
    df['julian_day'] = df['datetime'].dt.strftime('%j').astype(float)
    df['main_eq_time_decimal'] = df['hour'] + df['min'] / 60 + df['sec'] / 3600
    df['cos_hour'] = np.cos(2 * np.pi * df['main_eq_time_decimal'] / 24)
    df['sin_hour'] = np.sin(2 * np.pi * df['main_eq_time_decimal'] / 24)
    df['is_night'] = ((df['hour'] < 6) | (df['hour'] >= 22)).astype(int)
    df['hour_fraction'] = df['main_eq_time_decimal'] / 24
    df['season'] = ((df['month'] % 12 + 3) // 3).astype(int)
    df['is_weekend'] = df['datetime'].dt.weekday >= 5
    if is_train:
        df['distance_km'] = haversine(df['lat'], df['lon'], df['lat_as'], df['lon_as'])
        df['delta_lat'] = df['lat_as'] - df['lat']
        df['delta_lon'] = df['lon_as'] - df['lon']
        df['delta_depth'] = df['depth_as'] - df['depth']
    else:
        df['distance_km'] = train['distance_km'].mean()
        df['delta_lat'] = 0
        df['delta_lon'] = 0
        df['delta_depth'] = 0
    df = df.replace([np.inf, -np.inf], np.nan).fillna(0)
    return df

train = add_features(train, is_train=True)
test = add_features(test, is_train=False)

features = [
    "year", "month", "day", "hour", "min", "sec", "lat", "lon", "depth", "class",
    "day_of_year", "julian_day", "main_eq_time_decimal", "cos_hour", "sin_hour",
    "is_night", "distance_km", "hour_fraction", "season", "is_weekend",
    "delta_lat", "delta_lon", "delta_depth"
]

time_targets = ["year_as", "month_as", "day_as", "hour_as", "min_as", "sec_as"]
space_targets = ["lat_as", "lon_as", "depth_as", "class_as"]

y_time = train[time_targets].copy()
y_space = train[space_targets].copy()
X = train[features].copy()
X_test = test[features].copy()

scaler_X = StandardScaler()
scaler_y_time = StandardScaler()
scaler_y_space = StandardScaler()
X_scaled = scaler_X.fit_transform(X)
y_time_scaled = scaler_y_time.fit_transform(y_time)
y_space_scaled = scaler_y_space.fit_transform(y_space)
X_test_scaled = scaler_X.transform(X_test)

class PyTorchModel(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 128), nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(128, 64), nn.ReLU(),
            nn.Linear(64, output_size)
        )
    def forward(self, x):
        return self.model(x)

kf = KFold(n_splits=5, shuffle=True, random_state=42)
oof_time = np.zeros_like(y_time_scaled)
oof_space_tabnet = np.zeros_like(y_space_scaled)
oof_space_pytorch = np.zeros_like(y_space_scaled)

for fold, (train_idx, val_idx) in enumerate(kf.split(X_scaled)):
    print(f"\nFold {fold + 1}")
    X_train, X_val = X_scaled[train_idx], X_scaled[val_idx]
    y_train_time, y_val_time = y_time_scaled[train_idx], y_time_scaled[val_idx]
    y_train_space, y_val_space = y_space_scaled[train_idx], y_space_scaled[val_idx]

    cat_time = MultiOutputRegressor(CatBoostRegressor(verbose=0, task_type='GPU', iterations=500))
    cat_time.fit(X_train, y_train_time)
    oof_time[val_idx] = cat_time.predict(X_val)

    tabnet = TabNetRegressor(
        n_d=8, n_a=16, n_steps=2, gamma=1.2, lambda_sparse=1e-4,
        optimizer_fn=torch.optim.Adam, optimizer_params=dict(lr=0.01),
        verbose=0, seed=42
    )
    tabnet.fit(X_train, y_train_space, eval_set=[(X_val, y_val_space)], patience=30)
    oof_space_tabnet[val_idx] = tabnet.predict(X_val)

    model = PyTorchModel(input_size=X_train.shape[1], output_size=y_train_space.shape[1])
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    loss_fn = nn.MSELoss()
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train_space, dtype=torch.float32)
    X_val_tensor = torch.tensor(X_val, dtype=torch.float32)

    for epoch in range(50):
        model.train()
        optimizer.zero_grad()
        loss = loss_fn(model(X_train_tensor), y_train_tensor)
        loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        oof_space_pytorch[val_idx] = model(X_val_tensor).numpy()

X_meta = np.hstack([oof_time, oof_space_tabnet, oof_space_pytorch])
y_all_scaled = np.hstack([y_time_scaled, y_space_scaled])
meta_model = MultiOutputRegressor(RidgeCV())
meta_model.fit(X_meta, y_all_scaled)

cat_time.fit(X_scaled, y_time_scaled)

tabnet = TabNetRegressor(
    n_d=8, n_a=16, n_steps=2, gamma=1.2, lambda_sparse=1e-4,
    optimizer_fn=torch.optim.Adam, optimizer_params=dict(lr=0.01),
    verbose=0, seed=42
)
tabnet.fit(X_scaled, y_space_scaled)
preds_tabnet = tabnet.predict(X_test_scaled)

model = PyTorchModel(input_size=X_scaled.shape[1], output_size=y_space_scaled.shape[1])
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y_space_scaled, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)

for epoch in range(50):
    model.train()
    optimizer.zero_grad()
    loss = loss_fn(model(X_tensor), y_tensor)
    loss.backward()
    optimizer.step()

model.eval()
with torch.no_grad():
    preds_pytorch = model(X_test_tensor).numpy()

preds_time = cat_time.predict(X_test_scaled)
X_test_meta = np.hstack([preds_time, preds_tabnet, preds_pytorch])
final_preds_scaled = meta_model.predict(X_test_meta)
final_preds = np.hstack([
    scaler_y_time.inverse_transform(final_preds_scaled[:, :len(time_targets)]),
    scaler_y_space.inverse_transform(final_preds_scaled[:, len(time_targets):])
])

for i, col in enumerate(time_targets + space_targets):
    test[col] = final_preds[:, i]

test["year_as"] = test["year_as"].round().astype(int)
test["month_as"] = test["month_as"].round().astype(int)
test["day_as"] = test["day_as"].round().astype(int)
test["hour_as"] = test["hour_as"].round().astype(int)
test["min_as"] = test["min_as"].round().astype(int)
test["sec_as"] = test["sec_as"].round(3)
test["lat_as"] = test["lat_as"].round(5)
test["lon_as"] = test["lon_as"].round(5)
test["depth_as"] = test["depth_as"].round(2)
test["class_as"] = test["class_as"].round(2)

submission = test[["id_eq"] + time_targets + space_targets]
submission.to_csv("submission_tabnet_pytorch_catboost.csv", index=False)
print("\nsubmission_tabnet_pytorch_catboost.csv сохранён!")



Fold 1

Early stopping occurred at epoch 30 with best_epoch = 0 and best_val_0_mse = 3.36429


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



Fold 2

Early stopping occurred at epoch 30 with best_epoch = 0 and best_val_0_mse = 4.38157


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



Fold 3

Early stopping occurred at epoch 30 with best_epoch = 0 and best_val_0_mse = 4.05067


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



Fold 4


KeyboardInterrupt: 

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.linear_model import RidgeCV
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostRegressor
from pytorch_tabnet.tab_model import TabNetRegressor
import torch
import torch.nn as nn

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

if 'Unnamed: 0' in train.columns:
    train.drop(columns=['Unnamed: 0'], inplace=True)

def add_features(df):
    df['datetime'] = pd.to_datetime(dict(year=df['year'], month=df['month'], day=df['day'],
                                         hour=df['hour'], minute=df['min'], second=df['sec']), errors='coerce')
    df['day_of_year'] = df['datetime'].dt.dayofyear.fillna(0)
    df['main_eq_time_decimal'] = df['hour'] + df['min'] / 60 + df['sec'] / 3600
    df['cos_hour'] = np.cos(2 * np.pi * df['main_eq_time_decimal'] / 24)
    df['sin_hour'] = np.sin(2 * np.pi * df['main_eq_time_decimal'] / 24)
    df = df.replace([np.inf, -np.inf], np.nan).fillna(0)
    return df

train = add_features(train)
test = add_features(test)

features = [
    "year", "month", "day", "hour", "min", "sec", "lat", "lon", "depth", "class",
    "day_of_year", "main_eq_time_decimal", "cos_hour", "sin_hour"
]
targets = [
    "year_as", "month_as", "day_as", "hour_as", "min_as", "sec_as",
    "lat_as", "lon_as", "depth_as", "class_as"
]

X = train[features].copy()
y = train[targets].copy()
X_test = test[features].copy()

scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)
X_test_scaled = scaler_X.transform(X_test)

class PyTorchModel(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 128), nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(128, 64), nn.ReLU(),
            nn.Linear(64, output_size)
        )
    def forward(self, x):
        return self.model(x)

oof_preds_tabnet = np.zeros_like(y_scaled)
oof_preds_cat = np.zeros_like(y_scaled)
oof_preds_pytorch = np.zeros_like(y_scaled)

kf = KFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(kf.split(X_scaled)):
    print(f"Fold {fold+1}")
    X_train, X_val = X_scaled[train_idx], X_scaled[val_idx]
    y_train, y_val = y_scaled[train_idx], y_scaled[val_idx]

    tabnet = TabNetRegressor(n_d=16, n_a=123, n_steps=3, gamma=1.5, lambda_sparse=1e-4,
                              optimizer_fn=torch.optim.Adam, optimizer_params=dict(lr=0.009),
                              verbose=0)
    tabnet.fit(X_train, y_train, eval_set=[(X_val, y_val)], patience=30)
    oof_preds_tabnet[val_idx] = tabnet.predict(X_val)

    cat = MultiOutputRegressor(CatBoostRegressor(verbose=0, task_type='GPU', iterations=500))
    cat.fit(X_train, y_train)
    oof_preds_cat[val_idx] = cat.predict(X_val)

    model = PyTorchModel(input_size=X_train.shape[1], output_size=y_train.shape[1])
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    loss_fn = nn.MSELoss()

    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
    X_val_tensor = torch.tensor(X_val, dtype=torch.float32)

    for epoch in range(50):
        model.train()
        optimizer.zero_grad()
        loss = loss_fn(model(X_train_tensor), y_train_tensor)
        loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        preds = model(X_val_tensor).numpy()
        oof_preds_pytorch[val_idx] = preds

X_meta = np.hstack([oof_preds_tabnet, oof_preds_cat, oof_preds_pytorch])
meta_model = MultiOutputRegressor(RidgeCV())
meta_model.fit(X_meta, y_scaled)

tabnet.fit(X_scaled, y_scaled)
preds_tabnet = tabnet.predict(X_test_scaled)

cat.fit(X_scaled, y_scaled)
preds_cat = cat.predict(X_test_scaled)

X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y_scaled, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)

model = PyTorchModel(input_size=X_scaled.shape[1], output_size=y_scaled.shape[1])
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()
for epoch in range(50):
    model.train()
    optimizer.zero_grad()
    loss = loss_fn(model(X_tensor), y_tensor)
    loss.backward()
    optimizer.step()

model.eval()
with torch.no_grad():
    preds_pytorch = model(X_test_tensor).numpy()

X_test_meta = np.hstack([preds_tabnet, preds_cat, preds_pytorch])
final_preds_scaled = meta_model.predict(X_test_meta)
final_preds = scaler_y.inverse_transform(final_preds_scaled)

for i, col in enumerate(targets):
    test[col] = final_preds[:, i]

test["year_as"] = test["year_as"].round().astype(int)
test["month_as"] = test["month_as"].round().astype(int)
test["day_as"] = test["day_as"].round().astype(int)
test["hour_as"] = test["hour_as"].round().astype(int)
test["min_as"] = test["min_as"].round().astype(int)
test["sec_as"] = test["sec_as"].round(3)
test["lat_as"] = test["lat_as"].round(5)
test["lon_as"] = test["lon_as"].round(5)
test["depth_as"] = test["depth_as"].round(2)
test["class_as"] = test["class_as"].round(2)

submission = test[["id_eq"] + targets]
submission.to_csv("submission_stacked.csv", index=False)
print("\n submission_stacked.csv сохранён")


Fold 1

Early stopping occurred at epoch 30 with best_epoch = 0 and best_val_0_mse = 143.433


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Fold 2

Early stopping occurred at epoch 30 with best_epoch = 0 and best_val_0_mse = 109.80448


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Fold 3

Early stopping occurred at epoch 30 with best_epoch = 0 and best_val_0_mse = 151.03052


...

Fold 4

Early stopping occurred at epoch 30 with best_epoch = 0 and best_val_0_mse = 121.88463


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Fold 5

Early stopping occurred at epoch 30 with best_epoch = 0 and best_val_0_mse = 193.13645


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)



 submission_stacked.csv сохранён


In [ ]:
def create_datetime(df, prefix):
    return pd.to_datetime(dict(year=df[f'{prefix}year_as'],
                               month=df[f'{prefix}month_as'],
                               day=df[f'{prefix}day_as'],
                               hour=df[f'{prefix}hour_as'],
                               minute=df[f'{prefix}min_as'],
                               second=df[f'{prefix}sec_as']))
real_time = create_datetime(test, prefix='')
pred_time = create_datetime(submission, prefix='')

main_eq_time = create_datetime(test, prefix='main_')

real_delta = (real_time - main_eq_time).dt.total_seconds()
pred_delta = (pred_time - main_eq_time).dt.total_seconds()

# Подсчёт случаев, когда прогноз был позже реального времени афтершока
late_predictions = (pred_delta > real_delta).sum()
total_predictions = len(pred_delta)

# Вычисление процента случаев, когда прогнозы не дали возможности эвакуироваться
percentage_late = (late_predictions / total_predictions) * 100

# Вывод результата
print(f"Процент прогнозов, не дающих вовремя эвакуироваться: {percentage_late:.2f}%")

KeyError: 'main_year_as'

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.linear_model import RidgeCV
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostRegressor
from pytorch_tabnet.tab_model import TabNetRegressor
import torch
import torch.nn as nn

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

if 'Unnamed: 0' in train.columns:
    train.drop(columns=['Unnamed: 0'], inplace=True)

def add_features(df):
    df['datetime'] = pd.to_datetime(dict(year=df['year'], month=df['month'], day=df['day'],
                                         hour=df['hour'], minute=df['min'], second=df['sec']), errors='coerce')
    df['day_of_year'] = df['datetime'].dt.dayofyear.fillna(0)
    df['main_eq_time_decimal'] = df['hour'] + df['min'] / 60 + df['sec'] / 3600
    df['cos_hour'] = np.cos(2 * np.pi * df['main_eq_time_decimal'] / 24)
    df['sin_hour'] = np.sin(2 * np.pi * df['main_eq_time_decimal'] / 24)
    df = df.replace([np.inf, -np.inf], np.nan).fillna(0)
    return df

train = add_features(train)
test = add_features(test)

features = [
    "year", "month", "day", "hour", "min", "sec", "lat", "lon", "depth", "class",
    "day_of_year", "main_eq_time_decimal", "cos_hour", "sin_hour"
]
targets = [
    "year_as", "month_as", "day_as", "hour_as", "min_as", "sec_as",
    "lat_as", "lon_as", "depth_as", "class_as"
]

X = train[features].copy()
y = train[targets].copy()
X_test = test[features].copy()

scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)
X_test_scaled = scaler_X.transform(X_test)

class PyTorchModel(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 128), nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(128, 64), nn.ReLU(),
            nn.Linear(64, output_size)
        )
    def forward(self, x):
        return self.model(x)

oof_preds_tabnet = np.zeros_like(y_scaled)
oof_preds_cat = np.zeros_like(y_scaled)
oof_preds_pytorch = np.zeros_like(y_scaled)

kf = KFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(kf.split(X_scaled)):
    print(f"Fold {fold+1}")
    X_train, X_val = X_scaled[train_idx], X_scaled[val_idx]
    y_train, y_val = y_scaled[train_idx], y_scaled[val_idx]

    tabnet = TabNetRegressor(n_d=16, n_a=123, n_steps=3, gamma=1.5, lambda_sparse=1e-4,
                              optimizer_fn=torch.optim.Adam, optimizer_params=dict(lr=0.009),
                              verbose=0)
    tabnet.fit(X_train, y_train, eval_set=[(X_val, y_val)], patience=30)
    oof_preds_tabnet[val_idx] = tabnet.predict(X_val)

    cat = MultiOutputRegressor(CatBoostRegressor(verbose=0, task_type='GPU', iterations=500))
    cat.fit(X_train, y_train)
    oof_preds_cat[val_idx] = cat.predict(X_val)

    model = PyTorchModel(input_size=X_train.shape[1], output_size=y_train.shape[1])
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    loss_fn = nn.MSELoss()

    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
    X_val_tensor = torch.tensor(X_val, dtype=torch.float32)

    for epoch in range(50):
        model.train()
        optimizer.zero_grad()
        loss = loss_fn(model(X_train_tensor), y_train_tensor)
        loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        preds = model(X_val_tensor).numpy()
        oof_preds_pytorch[val_idx] = preds

X_meta = np.hstack([oof_preds_tabnet, oof_preds_cat, oof_preds_pytorch])
meta_model = MultiOutputRegressor(RidgeCV())
meta_model.fit(X_meta, y_scaled)

tabnet.fit(X_scaled, y_scaled)
preds_tabnet = tabnet.predict(X_test_scaled)

cat.fit(X_scaled, y_scaled)
preds_cat = cat.predict(X_test_scaled)

X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y_scaled, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)

model = PyTorchModel(input_size=X_scaled.shape[1], output_size=y_scaled.shape[1])
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()
for epoch in range(50):
    model.train()
    optimizer.zero_grad()
    loss = loss_fn(model(X_tensor), y_tensor)
    loss.backward()
    optimizer.step()

model.eval()
with torch.no_grad():
    preds_pytorch = model(X_test_tensor).numpy()

X_test_meta = np.hstack([preds_tabnet, preds_cat, preds_pytorch])
final_preds_scaled = meta_model.predict(X_test_meta)
final_preds = scaler_y.inverse_transform(final_preds_scaled)

for i, col in enumerate(targets):
    test[col] = final_preds[:, i]

test["year_as"] = test["year_as"].round().astype(int)
test["month_as"] = test["month_as"].round().astype(int)
test["day_as"] = test["day_as"].round().astype(int)
test["hour_as"] = test["hour_as"].round().astype(int)
test["min_as"] = test["min_as"].round().astype(int)
test["sec_as"] = test["sec_as"].round(3)
test["lat_as"] = test["lat_as"].round(5)
test["lon_as"] = test["lon_as"].round(5)
test["depth_as"] = test["depth_as"].round(2)
test["class_as"] = test["class_as"].round(2)

submission = test[["id_eq"] + targets]
submission.to_csv("submission_stacked.csv", index=False)
print("\n submission_stacked.csv сохранён! (MAE ≈ 20.73)")

true_aftershock = pd.to_datetime(dict(
    year=train["year_as"],
    month=train["month_as"],
    day=train["day_as"],
    hour=train["hour_as"],
    minute=train["min_as"],
    second=train["sec_as"].astype(int)
))

pred_aftershock = pd.to_datetime(dict(
    year=test["year_as"],
    month=test["month_as"],
    day=test["day_as"],
    hour=test["hour_as"],
    minute=test["min_as"],
    second=test["sec_as"].astype(int)
))

late_prediction_mask = pred_aftershock > true_aftershock[:len(pred_aftershock)].values
late_ratio = 100 * late_prediction_mask.sum() / len(pred_aftershock)
print(f"\n⏱ Модель предсказала афтершок позже реального в {late_ratio:.2f}% случаев, то есть шанс не выжить грубо говоря такой")

Fold 1

Early stopping occurred at epoch 30 with best_epoch = 0 and best_val_0_mse = 143.433


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Fold 2

Early stopping occurred at epoch 30 with best_epoch = 0 and best_val_0_mse = 109.80448


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Fold 3

Early stopping occurred at epoch 30 with best_epoch = 0 and best_val_0_mse = 151.03052


...

Fold 4

Early stopping occurred at epoch 30 with best_epoch = 0 and best_val_0_mse = 121.88463


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Fold 5

Early stopping occurred at epoch 30 with best_epoch = 0 and best_val_0_mse = 193.13645


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)



 submission_stacked.csv сохранён! (MAE ≈ 20.73)

⏱️ Модель предсказала афтершок позже реального в 100.00% случаев, то есть шанс не выжить грубо говоря такой


In [ ]:
!pip install numpy pandas scikit-learn catboost pytorch-tabnet torch --quiet

   ━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.8/99.2 MB 1.8 MB/s eta 0:00:46
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 71.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.9 MB/s eta 0:00:00
ERROR: THESE PACKAGES DO NOT MATCH THE HASHES F

In [ ]:
!pip install catboost pytorch-tabnet --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 1.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster

test = pd.read_csv("test.csv")
submission = pd.read_csv("submission_stacked (2).csv")

assert "id_eq" in submission.columns, "Нет колонки id_eq в submission!"
merged = pd.merge(test, submission, on="id_eq", suffixes=("", "_pred"))

m = folium.Map(location=[merged["lat"].mean(), merged["lon"].mean()], zoom_start=5)
cluster = MarkerCluster().add_to(m)

for _, row in merged.iterrows():

    folium.CircleMarker(
        location=[row["lat"], row["lon"]],
        radius=4,
        color='blue',
        fill=True,
        fill_opacity=0.5,
        tooltip=f"MAIN\nclass: {row['class']}"
    ).add_to(cluster)

    folium.CircleMarker(
        location=[row["lat_as"], row["lon_as"]],
        radius=4,
        color='green',
        fill=True,
        fill_opacity=0.6,
        tooltip=f"PREDICTED\nclass: {row['class_as']}"
    ).add_to(cluster)

    folium.PolyLine(
        locations=[[row["lat"], row["lon"]], [row["lat_as"], row["lon_as"]]],
        color="orange", weight=2, opacity=0.7
    ).add_to(m)

m.save("aftershock_map.html")
print(" Карта сохранена как 'aftershock_map.html' — открой для презентации!")

m


✅ Карта сохранена как 'aftershock_map.html' — открой для презентации!


In [ ]:
submission.columns

Index(['id_eq', 'year_as', 'month_as', 'day_as', 'hour_as', 'min_as', 'sec_as',
       'lat_as', 'lon_as', 'depth_as', 'class_as'],
      dtype='object')